In [1]:
%run "10_Data into model format.ipynb"


function: load_DProcessed_df(file), amex_metric(y_true, y_pred), file_get(dataset,version,dataset_list,num_list_inModel,ignore_list)
list:all_cols,cat_list,num_list , varible: Huge_data_path


In [22]:
collect_list= pd.read_csv('Data/p_important_fulllist.csv',index_col=0)
collect_list = collect_list.drop(['D_139_mean', 'D_139_first', 'D_103_max', 'D_103_first', 'D_103_mean', 'D_139_last'])
collect_list.to_csv('Data/p_important_fulllist.csv')

In [10]:
num_list_XXXize

['D_55_cen',
 'D_56_cen',
 'D_59_Q50std',
 'D_60_cen',
 'D_61_cen',
 'D_73_std',
 'D_76_std',
 'R_9_std',
 'S_11_Q50std',
 'S_5_std',
 'S_7_cen',
 'S_8_std',
 'S_9_std']

In [13]:
add_list = ['D_59_Q50std','S_11_Q50std','R_7_Q50std','B_17_Q50std','D_46_Q50std']
num_list = add_list


dataset = 'test'
data_df,label_df = data_process_loadfiles(dataset)
data_df = data_process_baseUpdate(data_df)
data_df, num_list_XXXize = data_process_itemXXXize(data_df)

req_item = add_list+['S_2_dt','customer_ID']
data_df = data_df[req_item]
num_list = add_list
temp_df = dataP_basestat(data_df,add_list)


data_df = data_df.sort_values('S_2_dt')# early to latest #,ascending = False) #latest to early 
data_df[num_list] = data_df[num_list].astype('float32')
data_last = data_df.groupby("customer_ID")[num_list].last() #lag_org
data_frist = data_df.groupby("customer_ID")[num_list].first() #for NA fill
order_agg = pd.DataFrame(index = data_last.index)

#for k in range(1,7):
for k in [1,3,6,'f']:
    print('lag'+str(k), end = "\r", flush=True)
    data_n = pd.DataFrame(index=data_last.index)
    if k == 'f':
        data_n = data_n.join(data_frist,on='customer_ID')
    else:
        data_n = data_n.join(data_df.groupby('customer_ID').nth(-k-1)[num_list],on='customer_ID')
        data_n[data_n.isnull()] = data_frist
    col_list = data_n.columns

    data_n[num_list] = data_last[num_list] - data_n[num_list]
    data_n.columns =  [str(col) + '_lag'+str(k)+'_sub' for col in col_list]
    order_agg = order_agg.join(data_n,on='customer_ID')

    data_n.columns =  col_list
    data_n[num_list] = data_n[num_list]/data_last[num_list]
    data_n.columns =  [str(col) + '_lag'+str(k)+'_prop' for col in col_list]
    order_agg = order_agg.join(data_n,on='customer_ID')
order_agg = order_agg.astype('float16')
temp_df = temp_df.join(order_agg,on='customer_ID')

output_df = pd.DataFrame(index = data_df['customer_ID'].unique()).rename_axis(index = 'customer_ID')
for j in [3,6,]:
    num_agg = data_df.sort_values('S_2_dt').groupby("customer_ID")[num_list].nth(range(j)).groupby("customer_ID").mean()
    std_agg = data_df.sort_values('S_2_dt').groupby("customer_ID")[num_list].nth(range(j)).groupby("customer_ID").std()
    data_last = data_df.groupby("customer_ID")[num_list].last() #lag_org
    diff_agg = pd.DataFrame()
    diff_agg = data_last - num_agg

    num_agg.columns = [str(col) + '_f'+str(j)+'avg' for col in num_agg.columns]
    diff_agg.columns = [str(col) + '_lag_diff_f'+str(j)+'avg' for col in diff_agg.columns]
    std_agg.columns = [str(col) + '_f'+str(j)+'std' for col in std_agg.columns]

    output_df = output_df.join(num_agg,on='customer_ID')
    output_df = output_df.join(diff_agg,on='customer_ID')
    output_df = output_df.join(std_agg,on='customer_ID')

for j in [-3,-6,]:
    num_agg = data_df.sort_values('S_2_dt').groupby("customer_ID")[num_list].nth(range(-1,j-1,-1)).groupby("customer_ID").mean()
    std_agg = data_df.sort_values('S_2_dt').groupby("customer_ID")[num_list].nth(range(-1,j-1,-1)).groupby("customer_ID").std()
    data_last = data_df.groupby("customer_ID")[num_list].last() #lag_org
    diff_agg = pd.DataFrame()
    diff_agg = data_last - num_agg

    num_agg.columns = [str(col) + '_f'+str(j)+'avg' for col in num_agg.columns]
    diff_agg.columns = [str(col) + '_lag_diff_f'+str(j)+'avg' for col in diff_agg.columns]
    std_agg.columns = [str(col) + '_f'+str(j)+'std' for col in std_agg.columns]

    output_df = output_df.join(num_agg,on='customer_ID')
    output_df = output_df.join(diff_agg,on='customer_ID')
    output_df = output_df.join(std_agg,on='customer_ID')
output_df = output_df#.reset_index()

temp_df = temp_df.join(output_df,on='customer_ID')
temp_df = temp_df.reset_index()
temp_df = temp_df.drop('index',axis=1)
temp_df.to_feather(Huge_data_path + 'refeature_items' +'_'+dataset+'.ftr')


In [6]:
data_df

,B_1,B_10,B_11,B_12,B_13,B_14,B_15,B_16,B_17,B_18,B_19,B_2,B_20,B_21,B_22,B_23,B_24,B_25,B_26,B_27,B_28,B_29,B_3,B_30,B_31,B_32,B_33,B_36,B_37,B_38,B_39,B_4,B_40,B_41,B_42,B_5,B_6,B_7,B_8,B_9,D_102,D_103,D_104,D_105,D_106,D_107,D_108,D_109,D_110,D_111,D_112,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,D_130,D_131,D_132,D_133,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,D_39,D_41,D_42,D_43,D_44,D_45,D_46,D_47,D_48,D_49,D_50,D_51,D_52,D_53,D_54,D_55,D_56,D_58,D_59,D_60,D_61,D_62,D_63,D_64,D_65,D_66,D_68,D_69,D_70,D_71,D_72,D_73,D_74,D_75,D_76,D_77,D_78,D_79,D_80,D_81,D_82,D_83,D_84,D_86,D_87,D_88,D_89,D_91,D_92,D_93,D_94,D_96,P_2,P_3,P_4,R_1,R_10,R_11,R_12,R_13,R_14,R_15,R_16,R_17,R_18,R_19,R_2,R_20,R_21,R_22,R_23,R_24,R_25,R_26,R_27,R_28,R_3,R_4,R_5,R_6,R_7,R_8,R_9,S_11,S_12,S_13,S_15,S_16,S_17,S_18,S_19,S_2,S_20,S_22,S_23,S_24,S_25,S_26,S_27,S_3,S_5,S_6,S_7,S_8,S_9,customer_ID,period_count,target,S_2_dt,S_2_dt_d,S_2_dt_m,S_2_dt_y,S_2_dt_wd,S_2_dt_sm,S_2_dt_m_in3,period_th,period_reverth,S_2_yearto2018Mar31,D_55_cen,D_56_cen,D_60_cen,D_61_cen,S_7_cen,D_73_std,D_76_std,R_9_std,S_5_std,S_8_std,S_9_std,D_59_Q50std,S_11_Q50std,R_7_Q50std,B_17_Q50std,D_46_Q50std
0,0.008728,0.096191,0.002768,0.148315,0.118103,0.018387,0.016357,0.007664,NaN,0.652832,0.008522,1.006836,0.004730,0.002644,0.004807,0.050873,0.004326,0.007729,0.000272,0.002310,0.084656,NaN,0.004707,0.0,1,0.006626,1.000977,0.009972,0.004570,2.0,NaN,0.080994,0.210083,0.006805,NaN,0.170654,0.063904,0.059418,0.006466,0.008209,0.766602,1.008789,1.004883,0.893555,NaN,0.669922,NaN,0.004326,NaN,NaN,1.007812,0.007874,1.0,0.238281,0.0,4.0,0.232178,0.236206,0.0,0.702148,0.434326,0.003057,0.686523,0.008743,1.0,1.002930,1.007812,1.000000,0.002052,0.005970,NaN,0.004345,NaN,NaN,NaN,NaN,NaN,0.002426,0.003706,0.003819,NaN,0.000569,0.000610,0.002674,0.001734,0.008774,NaN,NaN,0.000630,0.708984,0.358643,0.525391,0.255615,NaN,0.148682,1.335938,0.207275,NaN,1.001953,0.354492,0.151978,0.158569,0.063660,0.199585,0.308350,0.091064,CR,O,0.007126,NaN,6.0,0.009010,0.008339,0.119385,0.004803,NaN,0.080444,0.069092,NaN,NaN,0.001575,0.004238,0.004059,0.003532,0.506836,0.007042,0.000830,0.007084,NaN,NaN,0.002665,1.503906,1.005859,0.003569,0.008873,0.004951,0.938477,0.736328,0.007553,0.009224,0.007122,0.002457,1.009766,0.003820,0.000438,0.006451,0.005054,0.000198,0.008911,0.005177,0.006203,0.007782,0.002449,0.007481,0.006893,0.003948,0.003647,NaN,1.008789,0.001534,0.001423,0.008301,0.001882,0.008362,0.007561,0.001434,NaN,0.401611,0.271973,0.515137,0.108276,0.002272,0.008034,0.005722,0.002537,2017-03-09,0.009705,0.894043,0.135620,0.911133,0.974609,0.001244,0.676758,0.124023,0.023376,0.008324,0.161377,0.922852,0.065735,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,13,0,2017-03-09,9,3,2017,3,0,0,1.0,13.0,-1.059570,0.068166,-0.057058,-0.173780,-0.096843,-0.055128,NaN,NaN,NaN,-0.484637,-0.172506,-0.330133,-2.224609,0.933594,0.997070,NaN,-1.745117
1,0.004925,0.099792,0.002748,0.143555,0.118713,0.013039,0.017685,0.007149,NaN,0.646973,0.002237,1.000977,0.003880,0.004192,0.001283,0.040466,0.004204,0.001864,0.000978,0.001327,0.081848,NaN,0.002714,0.0,1,0.001854,1.006836,0.003922,0.004654,2.0,NaN,0.069397,0.184082,0.004406,NaN,0.113220,0.065247,0.057739,0.001614,0.008369,0.786133,1.000000,1.003906,0.906738,NaN,0.668457,NaN,0.008705,NaN,NaN,1.007812,0.003445,1.0,0.247192,0.0,4.0,0.243530,0.241943,0.0,0.707031,0.430420,0.001306,0.686523,0.000755,1.0,1.008789,1.003906,1.008789,0.001034,0.004837,NaN,0.007496,NaN,NaN,NaN,NaN,NaN,0.003956,0.003166,0.005032,NaN,0.009575,0.005493,0.009216,0.005775,0.000798,NaN,NaN,0.002525,0.712891,0.353516,0.521484,0.223389,NaN,0.149780,1.339844,0.202759,NaN,1.008789,0.326660,0.156250,0.148438,0.065491,0.151367,0.265137,0.086792,CR,O,0.002413,NaN,6.0,0.007843,0.006523,0.140625,0.000094,NaN,0.081421,0.074158,NaN,NaN,0.009895,0.007599,0.000126,0.007774,0.500977,0.007790,0.009468,0.006676,NaN,NaN,0.002508,1.

In [5]:
dataset = 'train'
data_df,label_df = data_process_loadfiles(dataset)
data_df = data_process_baseUpdate(data_df)
data_df, num_list_XXXize = data_process_itemXXXize(data_df)


KeyboardInterrupt: 

In [11]:
dataset = 'test'
data_df = pd.read_pickle(Huge_data_path + 'base_df_test.pkl')
applylist = [col for col in num_list+num_list_XXXize if col not in Q50diff+Q50diff2]
num_agg = data_df.sort_values('S_2_dt').groupby("customer_ID")[applylist].agg(['sum', 'min', 'max', 'count','mean'])

num_agg_proc = pd.DataFrame(index=num_agg.index)

for item in applylist:
    temp_t = num_agg[item][num_agg[item]['count']>5]
    temp_t= (temp_t['sum']-temp_t['min']-temp_t['max'])/temp_t['count']
    temp_t1  = num_agg[item][num_agg[item]['count']<=5]
    temp_t1 = temp_t1['mean']
    temp_t1.name = 0
    full_temp = pd.concat([temp_t,temp_t1])#.rename(columns={0:item+'_LmmMean'})
    num_agg_proc =  pd.concat([num_agg_proc,full_temp],axis=1).rename(columns={0:item+'_LmmMean'})
    print(item,end="\r", flush=True)
num_agg_proc = num_agg_proc.reset_index()
num_agg_proc.to_feather(Huge_data_path + 'LmmMean' +'_'+dataset+'.ftr')

ACF PACF

In [15]:
dataset = 'train'
data_df,label_df = data_process_loadfiles(dataset)
data_df = data_process_baseUpdate(data_df)
data_df, num_list_XXXize = data_process_itemXXXize(data_df)


In [28]:


#temp_df = data_df[['customer_ID','B_1']].groupby('customer_ID').count()
for item in num_list[5:]+num_list_XXXize:
#for item in ['B_1','D_119']:
    try:
        ex_df = data_df[[item,'customer_ID']].groupby('customer_ID')[item].apply(acf,nlags=1)
        aa = pd.DataFrame(ex_df).explode(item)
        aa = aa.groupby(aa.index).nth(1).rename(columns={item:item+'_acf'})
        temp_df = pd.concat([temp_df,aa],axis=1)
    except:
        1

    try:
        ex_df = data_df[[item,'customer_ID']].groupby('customer_ID')[item].apply(pacf)
        aa = pd.DataFrame(ex_df).explode(item)
        aa = aa.groupby(aa.index).nth(1).rename(columns={item:item+'_pacf'})
        temp_df = pd.concat([temp_df,aa],axis=1)
    except:
        1
    print(item,end="\r", flush=True)
temp_df = temp_df.drop('B_1',axis=1).reset_index()
temp_df.to_feather(Huge_data_path + 'acfpacf' +'_'+dataset+'.ftr')
temp_df

,customer_ID,B_1_acf,B_1_pacf,B_10_acf,B_10_pacf,B_11_acf,B_11_pacf,B_12_acf,B_12_pacf,B_13_acf,B_14_acf,B_14_pacf,B_15_acf,B_15_pacf,B_16_acf,B_16_pacf,B_17_acf,B_17_pacf,B_18_acf,B_19_acf,B_19_pacf,B_2_acf,B_2_pacf,B_20_acf,B_20_pacf,B_21_acf,B_22_acf,B_22_pacf,B_23_acf,B_23_pacf,B_24_acf,B_24_pacf,B_25_acf,B_25_pacf,B_26_acf,B_26_pacf,B_27_acf,B_27_pacf,B_28_acf,B_28_pacf,B_29_acf,B_29_pacf,B_3_acf,B_3_pacf,B_31_acf,B_32_acf,B_32_pacf,B_33_acf,B_36_acf,B_36_pacf,B_37_acf,B_37_pacf,B_39_acf,B_39_pacf,B_4_acf,B_4_pacf,B_40_acf,B_41_acf,B_41_pacf,B_42_acf,B_42_pacf,B_5_acf,B_5_pacf,B_6_acf,B_6_pacf,B_7_acf,B_7_pacf,B_8_acf,B_9_acf,B_9_pacf,D_102_acf,D_102_pacf,D_104_acf,D_104_pacf,D_105_acf,D_105_pacf,D_106_acf,D_106_pacf,D_107_acf,D_107_pacf,D_108_acf,D_108_pacf,D_109_acf,D_109_pacf,D_110_acf,D_110_pacf,D_111_acf,D_111_pacf,D_112_acf,D_113_acf,D_113_pacf,D_115_acf,D_115_pacf,D_118_acf,D_118_pacf,D_119_acf,D_119_pacf,D_121_acf,D_121_pacf,D_122_acf,D_122_pacf,D_123_acf,D_123_pacf,D_124_acf,D_124_pacf,D_125_acf,D_125_pacf,D_127_acf,D_127_pacf,D_128_acf,D_129_acf,D_129_pacf,D_130_acf,D_131_acf,D_131_pacf,D_132_acf,D_132_pacf,D_133_acf,D_133_pacf,D_134_acf,D_134_pacf,D_135_acf,D_135_pacf,D_136_acf,D_136_pacf,D_137_acf,D_137_pacf,D_138_acf,D_138_pacf,D_140_acf,D_140_pacf,D_141_acf,D_141_pacf,D_142_acf,D_142_pacf,D_143_acf,D_143_pacf,D_144_acf,D_144_pacf,D_145_acf,D_145_pacf,D_39_acf,D_39_pacf,D_41_acf,D_41_pacf,D_42_acf,D_42_pacf,D_43_acf,D_43_pacf,D_44_acf,D_44_pacf,D_45_acf,D_45_pacf,D_46_acf,D_46_pacf,D_47_acf,D_47_pacf,D_48_acf,D_48_pacf,D_49_acf,D_49_pacf,D_50_acf,D_50_pacf,D_51_acf,D_51_pacf,D_52_acf,D_52_pacf,D_53_acf,D_53_pacf,D_54_acf,D_55_acf,D_55_pacf,D_56_acf,D_56_pacf,D_58_acf,D_58_pacf,D_59_acf,D_59_pacf,D_60_acf,D_61_acf,D_61_pacf,D_62_acf,D_65_acf,D_65_pacf,D_69_acf,D_70_acf,D_70_pacf,D_71_acf,D_71_pacf,D_72_acf,D_72_pacf,D_73_acf,D_73_pacf,D_74_acf,D_74_pacf,D_75_acf,D_75_pacf,D_76_acf,D_76_pacf,D_77_acf,D_78_acf,D_78_pacf,D_79_acf,D_79_pacf,D_80_acf,D_80_pacf,D_81_acf,D_81_pacf,D_82_acf,D_82_pacf,D_83_acf,D_83_pacf,D_84_acf,D_84_pacf,D_86_acf,D_86_pacf,D_87_acf,D_88_acf,D_88_pacf,D_89_acf,D_89_pacf,D_91_acf,D_91_pacf,D_92_acf,D_92_pacf,D_93_acf,D_93_pacf,D_94_acf,D_94_pacf,D_96_acf,D_96_pacf,P_2_acf,P_2_pacf,P_3_acf,P_3_pacf,P_4_acf,P_4_pacf,R_1_acf,R_1_pacf,R_10_acf,R_10_pacf,R_11_acf,R_12_acf,R_13_acf,R_13_pacf,R_14_acf,R_14_pacf,R_15_acf,R_15_pacf,R_16_acf,R_16_pacf,R_17_acf,R_17_pacf,R_18_acf,R_18_pacf,R_19_acf,R_19_pacf,R_2_acf,R_2_pacf,R_20_acf,R_20_pacf,R_21_acf,R_21_pacf,R_22_acf,R_22_pacf,R_23_acf,R_23_pacf,R_24_acf,R_24_pacf,R_25_acf,R_25_pacf,R_26_acf,R_26_pacf,R_27_acf,R_28_acf,R_28_pacf,R_3_acf,R_3_pacf,R_4_acf,R_4_pacf,R_5_acf,R_5_pacf,R_6_acf,R_6_pacf,R_7_acf,R_7_pacf,R_8_acf,R_8_pacf,R_9_acf,R_9_pacf,S_11_acf,S_11_pacf,S_12_acf,S_12_pacf,S_13_acf,S_13_pacf,S_15_acf,S_15_pacf,S_16_acf,S_17_acf,S_17_pacf,S_18_acf,S_18_pacf,S_19_acf,S_19_pacf,S_20_acf,S_20_pacf,S_22_acf,S_23_acf,S_23_pacf,S_24_acf,S_24_pacf,S_25_acf,S_26_acf,S_26_pacf,S_27_acf,S_27_pacf,S_3_acf,S_3_pacf,S_5_acf,S_5_pacf,S_6_acf,S_7_acf,S_7_pacf,S_8_acf,S_8_pacf,S_9_acf,S_9_pacf,B_17_Q50std_acf,B_17_Q50std_pacf,D_46_Q50std_acf,D_46_Q50std_pacf,D_55_cen_acf,D_55_cen_pacf,D_56_cen_acf,D_56_cen_pacf,D_59_Q50std_acf,D_59_Q50std_pacf,D_60_cen_acf,D_60_cen_pacf,D_61_cen_acf,D_61_cen_pacf,D_73_std_acf,D_73_std_pacf,D_76_std_acf,D_76_std_pacf,R_7_Q50std_acf,R_7_Q50std_pacf,R_9_std_acf,R_9_std_pacf,S_11_Q50std_acf,S_11_Q50std_pacf,S_5_std_acf,S_5_std_pacf,S_7_cen_acf,S_7_cen_pacf,S_8_std_acf,S_8_std_pacf,S_9_std_acf,S_9_std_pacf
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,-0.169165,-0.183262,0.563356,0.610302,-0.003708,-0.004017,0.669339,0.725118,0.606859,-0.42926,-0.465031,-0.383402,-0.415353,-0.123413,-0.133697,NaN,NaN,0.770264,0.190479,0.206352,-0.231883,-0.251207,0.535382,0.579997,-0.207202,-0.231795,-0.251111,0.590431,0.639633,-0.250912,-0.271821,0.006176,0.006691,0.221742,0.24022,-0.028302,-0.030661,0.535138,0.579733,NaN,NaN,-0.24

In [3]:
dataset = 'test'
data_df,label_df = data_process_loadfiles(dataset)
data_df = data_process_baseUpdate(data_df)
data_df, num_list_XXXize = data_process_itemXXXize(data_df)

In [6]:
data_df.to_pickle(Huge_data_path + 'base_df' +'_'+dataset+'.pkl')

In [4]:
pacf_set = ['B_1','B_2','B_3','B_4','B_7','B_9','B_11','B_12','B_14','B_16','B_17','B_19','B_20','B_22','B_23','B_25','B_28','B_37','B_42','D_41','D_42','D_44','D_46','D_46_Q50std','D_48','D_53','D_55','D_58','D_59_Q50std','D_70','D_73_std','D_74','D_75','P_2','P_3','P_4','R_1','R_2','S_5','S_5_std','S_23','S_24']
acf_set = ['B_1','B_2','B_3','B_4','B_7','B_9','B_11','B_12','B_14','B_16','B_17','B_18','B_19','B_20','B_22','B_23','B_25','B_28','B_31','B_37','B_40','B_42','D_41','D_42','D_44','D_46','D_46_Q50std','D_48','D_53','D_55','D_58','D_59_Q50std','D_60','D_70','D_73_std','D_74','D_75','D_76','D_112','D_113','P_2','P_3','P_4','R_1','R_2','R_3','S_5','S_5_std','S_6','S_22','S_23','S_24','S_25','S_27']


In [5]:
aa = pd.read_feather(Huge_data_path + 'acfpacf' +'_'+'train'+'.ftr')
aa[temp_df.columns]

,customer_ID,B_1_acf,B_2_acf,B_3_acf,B_4_acf,B_7_acf,B_9_acf,B_11_acf,B_12_acf,B_14_acf,B_16_acf,B_17_acf,B_18_acf,B_19_acf,B_20_acf,B_22_acf,B_23_acf,B_25_acf,B_28_acf,B_31_acf,B_37_acf,B_40_acf,B_42_acf,D_41_acf,D_42_acf,D_44_acf,D_46_acf,D_46_Q50std_acf,D_48_acf,D_53_acf,D_55_acf,D_58_acf,D_59_Q50std_acf,D_60_acf,D_70_acf,D_73_std_acf,D_74_acf,D_75_acf,D_76_acf,D_112_acf,D_113_acf,P_2_acf,P_3_acf,P_4_acf,R_1_acf,R_2_acf,R_3_acf,S_5_acf,S_5_std_acf,S_6_acf,S_22_acf,S_23_acf,S_24_acf,S_25_acf,S_27_acf,B_1_pacf,B_2_pacf,B_3_pacf,B_4_pacf,B_7_pacf,B_9_pacf,B_11_pacf,B_12_pacf,B_14_pacf,B_16_pacf,B_17_pacf,B_19_pacf,B_20_pacf,B_22_pacf,B_23_pacf,B_25_pacf,B_28_pacf,B_37_pacf,B_42_pacf,D_41_pacf,D_44_pacf,D_46_pacf,D_46_Q50std_pacf,D_48_pacf,D_53_pacf,D_55_pacf,D_58_pacf,D_59_Q50std_pacf,D_70_pacf,D_73_std_pacf,D_74_pacf,D_75_pacf,P_2_pacf,P_3_pacf,R_1_pacf,R_2_pacf,S_5_pacf,S_5_std_pacf,S_23_pacf,S_24_pacf
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,-0.169165,-0.231883,-0.243439,0.574407,0.631489,-0.212771,-0.003708,0.669339,-0.429260,-0.123413,NaN,0.770264,0.190479,0.535382,-0.231795,0.590431,0.006176,0.535138,NaN,-0.353279,0.744738,NaN,-0.299893,NaN,-0.151704,0.555989,0.561293,NaN,NaN,0.757811,0.797918,0.535725,0.616245,0.139103,NaN,0.743856,0.773814,NaN,0.310667,-0.151328,0.162097,0.380995,0.352740,-0.157105,-0.224999,-0.157742,-0.002148,-0.277718,0.233324,-0.351648,-0.170060,-0.420699,-0.196712,0.350814,-0.183262,-0.251207,-0.263725,0.622274,0.684114,-0.230502,-0.004017,0.725118,-0.465031,-0.133697,NaN,0.206352,0.579997,-0.251111,0.639633,0.006691,0.579733,-0.382719,NaN,-0.324885,-0.164346,0.602322,0.608067,NaN,NaN,0.820962,0.864411,0.580368,0.150695,NaN,0.805844,0.838299,0.175605,0.412744,-0.170197,-0.243749,-0.002327,-0.300861,-0.184231,-0.455757
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.014104,-0.064961,-0.456659,-0.063415,-0.067327,-0.086081,-0.012306,0.530710,0.009870,0.280930,NaN,0.226562,-0.138551,-0.490318,-0.354561,-0.112237,-0.022995,-0.138537,NaN,0.128171,0.249759,NaN,-0.297201,NaN,0.154732,0.403737,0.378060,0.068523,NaN,0.565026,0.245076,0.581196,0.047347,0.165125,NaN,-0.045415,0.292833,NaN,-0.433791,-0.131817,0.635774,0.474065,0.106236,0.029689,-0.198490,0.763857,0.127156,0.524463,-0.290912,-0.274724,0.081320,-0.110507,-0.280347,NaN,0.015279,-0.070374,-0.494713,-0.068699,-0.072938,-0.093255,-0.013332,0.574935,0.010693,0.304341,NaN,-0.150097,-0.531178,-0.384107,-0.121590,-0.024911,-0.150082,0.138852,NaN,-0.321967,0.167627,0.437382,0.409565,0.074233,NaN,0.612111,0.265499,0.629629,0.178885,NaN,-0.049199,0.317236,0.688755,0.513570,0.032163,-0.215031,0.137753,0.568168,0.088097,-0.119715
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.162754,-0.276336,-0.327187,-0.221848,-0.201583,-0.280244,-0.187909,-0.012901,-0.782404,0.044663,NaN,0.638926,-0.082982,0.239287,-0.206886,-0.233473,0.248029,-0.263192,NaN,0.110272,-0.309189,NaN,0.037532,NaN,-0.107274,-0.119189,-0.115048,0.148087,NaN,0.628649,0.673816,0.583596,-0.014421,-0.531055,NaN,0.605269,0.655437,NaN,-0.117635,0.149684,-0.083051,-0.108443,-0.174607,0.177738,-0.112655,-0.291610,0.170029,-0.357183,-0.566782,0.121601,-0.345794,0.086133,-0.314959,NaN,0.176317,-0.299364,-0.354453,-0.240336,-0.218382,-0.303597,-0.203568,-0.013976,-0.847605,0.048385,NaN,-0.089897,0.259227,-0.224127,-0.252929,0.268698,-0.285124,0.119461,NaN,0.040660,-0.116214,-0.129121,-0.124635,0.160428,NaN,0.681037,0.729967,0.632229,-0.575310,NaN,0.655708,0.710056,-0.089971,-0.117480,0.192549,-0.122043,0.184198,-0.386948,-0.374610,0.093311
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.545207,0.161002,-0.415922,0.326368,0.325826,0.452532,0.482696,0.816216,0.499250,-0.353493,NaN,0.564760,-0.065236,-0.333036,0.019039,0.293765,0.476346,0.421320,NaN,0.519997,0.223119,NaN,0.139505,NaN,-0.564625,0.406391,0.396332,NaN,0.367128,0.492440,0.553098,0.608700,0.487614,-0.332003,NaN,0.559417,0.538695,NaN,0.753231,-0.038277,0.732285,-0.129609,0.386886,0.156685,-0.056849,0.165485,-0.004659

In [4]:
%run "10_Data into model format.ipynb"
dataset = 'test'
data_df =  pd.read_pickle(Huge_data_path + 'base_df' +'_'+dataset+'.pkl')
temp_df = data_df[['customer_ID','B_1']].groupby('customer_ID').count()
for item in acf_set:
    try:
        ex_df = data_df[[item,'customer_ID']].groupby('customer_ID')[item].apply(acf,nlags=1)
        aa = pd.DataFrame(ex_df).explode(item)
        aa = aa.groupby(aa.index).nth(1).rename(columns={item:item+'_acf'})
        temp_df = pd.concat([temp_df,aa],axis=1)
        print(item + '_acf',end="\r", flush=True)
    except:
        1
for item in pacf_set:
    try:
        ex_df = data_df[[item,'customer_ID']].groupby('customer_ID')[item].apply(pacf)
        aa = pd.DataFrame(ex_df).explode(item)
        aa = aa.groupby(aa.index).nth(1).rename(columns={item:item+'_pacf'})
        temp_df = pd.concat([temp_df,aa],axis=1)
        print(item + '_pacf',end="\r", flush=True)
    except:
        1
temp_df = temp_df.drop('B_1',axis=1).reset_index()
temp_df.to_feather(Huge_data_path + 'acfpacf' +'_'+dataset+'.ftr')
temp_df

function: load_DProcessed_df(file), amex_metric(y_true, y_pred), file_get(dataset,version,dataset_list,num_list_inModel,ignore_list)
list:all_cols,cat_list,num_list , varible: Huge_data_path


,customer_ID,B_1_acf,B_2_acf,B_3_acf,B_4_acf,B_7_acf,B_9_acf,B_11_acf,B_12_acf,B_14_acf,B_16_acf,B_17_acf,B_18_acf,B_19_acf,B_20_acf,B_22_acf,B_23_acf,B_25_acf,B_28_acf,B_31_acf,B_37_acf,B_40_acf,B_42_acf,D_41_acf,D_42_acf,D_44_acf,D_46_acf,D_46_Q50std_acf,D_48_acf,D_53_acf,D_55_acf,D_58_acf,D_59_Q50std_acf,D_60_acf,D_70_acf,D_73_std_acf,D_74_acf,D_75_acf,D_76_acf,D_112_acf,D_113_acf,P_2_acf,P_3_acf,P_4_acf,R_1_acf,R_2_acf,R_3_acf,S_5_acf,S_5_std_acf,S_6_acf,S_22_acf,S_23_acf,S_24_acf,S_25_acf,S_27_acf,B_1_pacf,B_2_pacf,B_3_pacf,B_4_pacf,B_7_pacf,B_9_pacf,B_11_pacf,B_12_pacf,B_14_pacf,B_16_pacf,B_17_pacf,B_19_pacf,B_20_pacf,B_22_pacf,B_23_pacf,B_25_pacf,B_28_pacf,B_37_pacf,B_42_pacf,D_41_pacf,D_44_pacf,D_46_pacf,D_46_Q50std_pacf,D_48_pacf,D_53_pacf,D_55_pacf,D_58_pacf,D_59_Q50std_pacf,D_70_pacf,D_73_std_pacf,D_74_pacf,D_75_pacf,P_2_pacf,P_3_pacf,R_1_pacf,R_2_pacf,S_5_pacf,S_5_std_pacf,S_23_pacf,S_24_pacf
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.102271,-0.222083,-0.018056,0.104695,0.057451,-0.643408,-0.147229,0.606235,0.035289,-0.181346,NaN,-0.299984,-0.136741,0.37042,-0.433606,0.056986,-0.213726,0.153441,NaN,-0.295072,0.134349,NaN,-0.181535,0.253438,-0.174057,NaN,NaN,0.641983,NaN,0.623346,-0.027756,NaN,0.246806,NaN,0.004806,-0.114358,-0.270349,NaN,-0.244658,NaN,-0.223207,NaN,-0.59834,-0.222645,0.423371,0.045398,-0.043956,-0.29528,0.469442,0.246968,-0.183434,0.197121,0.02474,0.263036,0.115055,-0.249844,-0.020313,0.117782,0.064632,-0.723834,-0.165633,0.682014,0.039701,-0.204014,NaN,-0.153833,0.416722,-0.487807,0.064109,-0.240441,0.172621,-0.331956,NaN,-0.204227,-0.195814,NaN,NaN,0.72223,NaN,0.701264,-0.031225,NaN,NaN,0.005407,-0.128653,-0.304143,-0.251108,NaN,-0.250476,0.476292,-0.04945,-0.33219,-0.206364,0.221761
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,-0.076968,-0.463857,-0.33447,-0.244506,-0.080525,-0.24987,-0.108511,0.701934,-0.281334,-0.451527,NaN,0.1431,-0.543065,0.436448,0.310598,-0.160232,-0.271645,-0.368457,NaN,-0.076973,0.085261,NaN,-0.116919,NaN,0.035341,-0.051271,-0.001686,0.245909,NaN,0.506116,0.008978,0.110861,0.075566,0.438525,NaN,-0.073684,0.121454,NaN,-0.225018,-0.379081,-0.008075,-0.038613,-0.213353,-0.265279,0.056965,0.203326,-0.109025,-0.208943,-0.070711,0.011748,-0.37577,0.290467,-0.3504,0.43582,-0.083382,-0.502512,-0.362342,-0.264882,-0.087235,-0.270693,-0.117553,0.760429,-0.304778,-0.489154,NaN,-0.58832,0.472819,0.336481,-0.173585,-0.294282,-0.399162,-0.083387,NaN,-0.126663,0.038287,-0.055544,-0.001826,0.266401,NaN,0.548292,0.009726,0.120099,0.475069,NaN,-0.079824,0.131576,-0.008748,-0.04183,-0.287386,0.061712,-0.11811,-0.226354,-0.407084,0.314673
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.20981,0.159097,-0.14481,0.77395,0.71829,-0.056709,0.104607,-0.298115,0.317016,0.499719,NaN,0.420468,0.070415,-0.378307,0.075167,0.71577,0.064271,0.774532,NaN,0.149529,0.719758,NaN,-0.107322,NaN,-0.080562,0.630939,0.621192,0.660702,NaN,0.632743,0.643053,0.661465,0.392832,0.431033,NaN,0.411193,0.452191,NaN,0.394346,-0.564856,0.681715,0.624045,-0.081193,-0.105879,0.063435,0.632448,0.081038,-0.006345,-0.029109,-0.048214,0.181233,-0.047103,-0.202802,0.4543,0.227294,0.172356,-0.156878,0.838446,0.778148,-0.061435,0.113324,-0.322958,0.343434,0.541362,NaN,0.076283,-0.409832,0.081431,0.775418,0.069627,0.839076,0.16199,NaN,-0.116266,-0.087276,0.683518,0.672958,0.715761,NaN,0.685472,0.696641,0.716587,0.466952,NaN,0.445459,0.489873,0.738525,0.676049,-0.114702,0.068721,0.087792,-0.006874,0.196336,-0.051029
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.161513,0.157263,0.263924,0.173942,0.35656,0.172078,0.154429,0.383588,-0.020703,-0.184643,0.191629,0.382902,-0.366844,0.076087,-0.109499,0.291661,0.180225,-0.032357,NaN,0.152722,0.348851,NaN,0.456711,NaN,0.492486,-0.11903,-0.087373,0.138751,NaN,0.758699,0.719823,-0.044049,0.320095,0.346701,NaN,0.748964,0.753406,NaN,0.356388,0.388763,0.263184,0.310545,-0.131809,0.593157,0.147592,0.292878,0.796588,0.74031,0.240517,0.367797,-0.35498,-0.058583,-0.54

In [29]:
dataset = 'test'
data_df,label_df = data_process_loadfiles(dataset)
data_df = data_process_baseUpdate(data_df)
data_df, num_list_XXXize = data_process_itemXXXize(data_df)
temp_df = data_df[['customer_ID','B_1']].groupby('customer_ID').count()
for item in num_list+num_list_XXXize:
#for item in ['B_1','D_119']:
    try:
        ex_df = data_df[[item,'customer_ID']].groupby('customer_ID')[item].apply(acf,nlags=1)
        aa = pd.DataFrame(ex_df).explode(item)
        aa = aa.groupby(aa.index).nth(1).rename(columns={item:item+'_acf'})
        temp_df = pd.concat([temp_df,aa],axis=1)
    except:
        1

    try:
        ex_df = data_df[[item,'customer_ID']].groupby('customer_ID')[item].apply(pacf)
        aa = pd.DataFrame(ex_df).explode(item)
        aa = aa.groupby(aa.index).nth(1).rename(columns={item:item+'_pacf'})
        temp_df = pd.concat([temp_df,aa],axis=1)
    except:
        1
    print(item,end="\r", flush=True)
temp_df = temp_df.drop('B_1',axis=1).reset_index()
temp_df.to_feather(Huge_data_path + 'acfpacf' +'_'+dataset+'.ftr')
temp_df

In [ ]:
add_list = ['D_59_Q50std','S_11_Q50std','R_7_Q50std','B_17_Q50std','D_46_Q50std']
num_list = add_list

dataset = 'train'
data_df,label_df = data_process_loadfiles(dataset)
data_df = data_process_baseUpdate(data_df)


medianvalue = data_df[Q50diff2+['S_2_dt']].groupby(['S_2_dt']).agg(['median'])
medianvalue.columns = ['_'.join(x) for x in medianvalue.columns]

data_df[Q50diff2] = data_df[Q50diff2].astype('float32')
Q75value = data_df[Q50diff2+['S_2_dt']].groupby(['S_2_dt']).quantile(0.75)
#Q75value.columns = [item+'_Q75']
data_df =data_df.merge(medianvalue, how='left', on=['S_2_dt'])
data_df =data_df.merge(Q75value, how='left', on=['S_2_dt'],suffixes=('', '_Q75'))
for item in Q50diff2:
    data_df[item+'_Q50std'] = (data_df[item] - data_df[item+'_median'])/(data_df[item+'_Q75']-data_df[item+'_median'])
    data_df = data_df.drop([item+'_median',item+'_Q75'],axis=1)
    data_df[item+'_Q50std']=data_df[item+'_Q50std'].astype('float16')
    num_list_XXXize.append(item+'_Q50std')


NameError: name 'num_list_XXXize' is not defined

In [ ]:
dataset = 'test'
data_df,label_df = data_process_loadfiles(dataset)
data_df = data_process_baseUpdate(data_df)
data_df, num_list_XXXize = data_process_itemXXXize(data_df)


temp_df = data_df[['customer_ID','B_1']].groupby('customer_ID').count()
for item in num_list+num_list_XXXize:
#for item in ['B_1','D_119']:
    ex_df = data_df[[item,'customer_ID']].groupby('customer_ID')[item].apply(acf,nlags=1)
    aa = pd.DataFrame(ex_df).explode(item)
    aa = aa.groupby(aa.index).nth(1).rename(columns={item:item+'_acf'})
    temp_df = pd.concat([temp_df,aa],axis=1)

    ex_df = data_df[[item,'customer_ID']].groupby('customer_ID')[item].apply(pacf)
    aa = pd.DataFrame(ex_df).explode(item)
    aa = aa.groupby(aa.index).nth(1).rename(columns={item:item+'_pacf'})
    temp_df = pd.concat([temp_df,aa],axis=1)
temp_df = temp_df.drop('B_1',axis=1).reset_index()
temp_df.to_feather(Huge_data_path + 'acfpacf' +'_'+dataset+'.ftr')
temp_df

Gen data from full set

In [ ]:
dataset = 'train'
collect_list= pd.read_csv('Data/p_important_fulllist.csv',index_col=0)
temp = pd.read_feather(Huge_data_path + 'package/time_WoE_baseset' +'_'+dataset+'.ftr').set_index('customer_ID')
collect_list = pd.concat([collect_list,pd.DataFrame({'r':np.repeat(0.0001,14),'dataset_word':np.repeat('time_WoE_baseset__',14)},index=temp.columns)])
temp = pd.read_pickle(Huge_data_path + 'package/time_stat' +'_'+dataset+'.plk')
collect_list = pd.concat([collect_list,pd.DataFrame({'r':np.repeat(0.0001,4),'dataset_word':np.repeat('time_stat__',4)},index=temp.columns)])
collect_list.to_csv('Data/p_important_fulllist.csv')

In [4]:
dataset = 'test'
data_df,label_df = data_process_loadfiles(dataset)
data_df = data_process_baseUpdate(data_df)
data_df, num_list_XXXize = data_process_itemXXXize(data_df)
#num_list = num_list_inModel+num_list_XXXize
#save data df - save feather need convert category

In [18]:
dataset = 'train'
temp_df = pd.read_feather(Huge_data_path + 'package/time_WoE_baseset' +'_'+dataset+'.ftr')
temp_df.columns

Index(['customer_ID', 'S_2_dt_wd_mean_WoE', 'S_2_dt_wd_std_WoE',
       'S_2_dt_wd_first_WoE', 'S_2_dt_wd_last_WoE', 'S_2_dt_wd_min_WoE',
       'S_2_dt_wd_max_WoE', 'S_2_dt_m_in3_mean_WoE', 'S_2_dt_m_in3_std_WoE',
       'S_2_dt_m_in3_first_WoE', 'S_2_dt_m_in3_last_WoE',
       'S_2_dt_m_in3_min_WoE', 'S_2_dt_m_in3_max_WoE', 'S_2_dt_wd_lagmean_WoE',
       'S_2_dt_m_in3_lagmean_WoE'],
      dtype='object')

In [11]:
aa = data_df[['S_2_dt_m','customer_ID']].groupby('customer_ID').last()[['S_2_dt_m']]
aa.to_pickle('Data/testdf_lastm.pkl')

In [2]:
#WoE Cat
time_list = ['S_2_dt_wd','S_2_dt_m_in3']
dataset = 'train'
data_df,label_df = data_process_loadfiles(dataset)
data_df = data_process_baseUpdate(data_df)
for item in time_list:
        NA_count = data_df[item].isna().sum()
        if NA_count >0:
            #NA_dict[item+str('-in_cat')] = NA_count
            #print(item)
            data_df[item] = data_df[item].cat.add_categories("NA").fillna("NA")


rare_encoder = RareLabelEncoder(tol=0.01, n_categories=2, variables=time_list,replace_with='Rare')

# set up a weight of evidence encoder
data_df_review = rare_encoder.fit_transform(data_df[time_list])
woe_encoder = WoEEncoder(variables=time_list)
woe_encoder.fit(data_df_review, data_df['target'])
train_t = woe_encoder.transform(data_df_review)
train_t[['customer_ID','S_2_dt']] = data_df[['customer_ID','S_2_dt']]

temp_df = dataP_basestat(train_t,time_list)
temp_df.columns = [col + '_WoE' if col!='customer_ID' else col for col in temp_df.columns ]
temp_df.to_feather(Huge_data_path + 'time_WoE_baseset' +'_'+dataset+'.ftr')
temp_df

,customer_ID,S_2_dt_wd_mean_WoE,S_2_dt_wd_std_WoE,S_2_dt_wd_first_WoE,S_2_dt_wd_last_WoE,S_2_dt_wd_min_WoE,S_2_dt_wd_max_WoE,S_2_dt_m_in3_mean_WoE,S_2_dt_m_in3_std_WoE,S_2_dt_m_in3_first_WoE,S_2_dt_m_in3_last_WoE,S_2_dt_m_in3_min_WoE,S_2_dt_m_in3_max_WoE,S_2_dt_wd_lagmean_WoE,S_2_dt_m_in3_lagmean_WoE
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.017685,0.037506,0.007278,0.008545,-0.013000,0.081848,0.002050,0.016754,0.022324,-0.011711,-0.011711,0.022324,-0.009148,-0.013756
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.027084,0.046295,0.004314,0.081848,-0.038635,0.081848,-0.001848,0.013916,0.022324,-0.007362,-0.011711,0.022324,0.054749,-0.005512
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,-0.013130,0.019165,-0.038635,-0.013000,-0.038635,0.008545,-0.006474,0.012779,-0.011711,-0.011711,-0.011711,0.022324,0.000130,-0.005238
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.000348,0.039368,-0.006256,0.007278,-0.038635,0.081848,-0.003798,0.011742,-0.007362,-0.007362,-0.011711,0.022324,0.006931,-0.003565
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,-0.001163,0.031281,0.007278,-0.006256,-0.038635,0.081848,-0.005077,0.008232,-0.007362,-0.007362,-0.007362,0.022324,-0.005096,-0.002283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,-0.000328,0.029449,-0.013000,-0.038635,-0.038635,0.081848,0.002384,0.016495,-0.011711,-0.007362,-0.011711,0.022324,-0.038330,-0.009750
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,-0.005768,0.016937,0.004314,0.007278,-0.038635,0.008545,-0.005413,0.008423,-0.007362,-0.007362,-0.011711,0.022324,0.013046,-0.001949
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,0.015205,0.041290,0.007278,0.004314,-0.038635,0.081848,0.007286,0.016968,-0.011711,0.022324,-0.011711,0.022324,-0.010887,0.015038
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,0.009071,0.034821,-0.038635,-0.006256,-0.038635,0.081848,-0.006752,0.008995,-0.007362,-0.007362,-0.011711,0.022324,-0.015327,-0.000612


In [3]:
#WoE Cat test
dataset = 'test'
test_df,label_df = data_process_loadfiles(dataset)
test_df = data_process_baseUpdate(test_df)

for item in time_list:
        NA_count = test_df[item].isna().sum()
        if NA_count >0:
            #NA_dict[item+str('-in_cat')] = NA_count
            #print(item)
            test_df[item] = test_df[item].cat.add_categories("NA").fillna("NA")
test_df[time_list] = rare_encoder.transform(test_df[time_list])
test_t = woe_encoder.transform(test_df[time_list])
test_t[['customer_ID','S_2_dt']] = test_df[['customer_ID','S_2_dt']]
temp_df = dataP_basestat(test_t,time_list)
temp_df.columns = [col + '_WoE' if col!='customer_ID' else col for col in temp_df.columns ]
temp_df.to_feather(Huge_data_path + 'cat_WoE_baseset' +'_'+dataset+'.ftr')
temp_df

ValueError: Length mismatch: Expected axis has 15 elements, new values have 14 elements

In [19]:

for dataset in ['train','test']:
    data_df,label_df = data_process_loadfiles(dataset)
    data_df = data_process_baseUpdate(data_df)
    catCust_agg = data_df.groupby("customer_ID")[time_list].agg(['last', 'nunique'])
    catCust_agg.columns = ['_'.join(x) for x in catCust_agg.columns]
    catCust_agg.to_pickle(Huge_data_path + 'time_stat' +'_'+dataset+'.plk')
    catCust_agg

In [5]:
for dataset in ['train','test']:
    data_df,label_df = data_process_loadfiles(dataset)
    catCust_agg = data_df.groupby("customer_ID")[cat_list].agg(['last', 'nunique'])
    catCust_agg.columns = ['_'.join(x) for x in catCust_agg.columns]
    catCust_agg.to_pickle(Huge_data_path + 'cat_stat' +'_'+dataset+'.ftr')
    catCust_agg

In [7]:
catCust_agg = pd.read_pickle(Huge_data_path + 'cat_stat' +'_'+dataset+'.ftr')
catCust_agg

,B_30_last,B_30_nunique,B_38_last,B_38_nunique,D_114_last,D_114_nunique,D_116_last,D_116_nunique,D_117_last,D_117_nunique,D_120_last,D_120_nunique,D_126_last,D_126_nunique,D_63_last,D_63_nunique,D_64_last,D_64_nunique,D_66_last,D_66_nunique,D_68_last,D_68_nunique
customer_ID,,,,,,,,,,,,,,,,,,,,,,
00000469ba478561f23a92a868bd366de6f6527a684c9a2e78fb826dcac3b9b7,0.0,1,2.0,2,0.0,1,0.0,1,-1.0,1,1.0,1,0.0,1,CR,1,U,1,NaN,0,6.0,1
00001bf2e77ff879fab36aa4fac689b9ba411dae63ae397d4263dafa1daedef5,0.0,1,2.0,2,1.0,1,0.0,1,3.0,1,0.0,1,1.0,1,CO,1,O,1,NaN,0,6.0,1
0000210045da4f81e5f122c6bde5c2a617d03eef67f82c5e400fc98e7bd43ce8,0.0,1,2.0,1,0.0,1,0.0,1,3.0,2,0.0,1,0.0,2,CR,1,U,2,1.0,1,4.0,2
00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976cf6e56734528702d694,0.0,2,3.0,1,0.0,1,0.0,1,4.0,3,0.0,2,1.0,1,CL,1,R,1,NaN,0,5.0,1
00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9a4693dd914fca22557,1.0,2,6.0,3,0.0,1,0.0,1,-1.0,1,1.0,2,0.0,1,CO,1,R,2,NaN,0,5.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c5d60460dba6dedc41e,0.0,1,1.0,1,1.0,1,0.0,1,3.0,1,0.0,1,1.0,1,CO,1,O,1,NaN,0,6.0,1
ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3a4f0ca3de613b0b2ad,0.0,1,3.0,2,1.0,1,0.0,1,3.0,1,0.0,2,1.0,1,CO,1,O,1,NaN,0,6.0,2
ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475cb095d2443a68030f1,0.0,1,3.0,1,1.0,1,0.0,1,3.0,1,0.0,2,1.0,1,CR,1,O,1,NaN,0,6.0,1


In [8]:
data_df[cat_list].isna().sum()

B_30     0
B_38     0
D_114    0
D_116    0
D_117    0
D_120    0
D_126    0
D_63     0
D_64     0
D_66     0
D_68     0
dtype: int64

In [16]:
data_df['S_2_dt']

0         2017-03-09
1         2017-04-07
2         2017-05-28
3         2017-06-13
4         2017-07-16
             ...    
5531446   2017-11-05
5531447   2017-12-23
5531448   2018-01-06
5531449   2018-02-06
5531450   2018-03-14
Name: S_2_dt, Length: 5531451, dtype: datetime64[ns]

In [31]:
#WoE Cat
dataset = 'train'
data_df,label_df = data_process_loadfiles(dataset)
data_df = data_process_baseUpdate(data_df)
for item in cat_list:
        NA_count = data_df[item].isna().sum()
        if NA_count >0:
            #NA_dict[item+str('-in_cat')] = NA_count
            #print(item)
            data_df[item] = data_df[item].cat.add_categories("NA").fillna("NA")


rare_encoder = RareLabelEncoder(tol=0.01, n_categories=2, variables=cat_list,replace_with='Rare')

# set up a weight of evidence encoder
data_df_review = rare_encoder.fit_transform(data_df[cat_list])
woe_encoder = WoEEncoder(variables=cat_list)
woe_encoder.fit(data_df_review, data_df['target'])
train_t = woe_encoder.transform(data_df_review)
train_t[['customer_ID','S_2_dt']] = data_df[['customer_ID','S_2_dt']]

temp_df = dataP_basestat(train_t,cat_list)
temp_df.columns = [col + '_WoE' if col!='customer_ID' else col for col in temp_df.columns ]
temp_df.to_feather(Huge_data_path + 'cat_WoE_baseset' +'_'+dataset+'.ftr')
temp_df


ValueError: Length mismatch: Expected axis has 45 elements, new values have 44 elements

In [21]:
#WoE Cat test
dataset = 'test'
test_df,label_df = data_process_loadfiles(dataset)
test_df = data_process_baseUpdate(test_df)

for item in cat_list:
        NA_count = test_df[item].isna().sum()
        if NA_count >0:
            #NA_dict[item+str('-in_cat')] = NA_count
            #print(item)
            test_df[item] = test_df[item].cat.add_categories("NA").fillna("NA")
test_df[cat_list] = rare_encoder.transform(test_df[cat_list])
test_t = woe_encoder.transform(test_df[cat_list])
test_t[['customer_ID','S_2_dt']] = test_df[['customer_ID','S_2_dt']]
temp_df = dataP_basestat(test_t,cat_list)
temp_df.columns = [col + '_WoE' for col in temp_df.columns if col!='customer_ID']
temp_df.to_feather(Huge_data_path + 'cat_WoE_baseset' +'_'+dataset+'.ftr')
temp_df


,customer_ID_WoE,B_30_mean_WoE,B_30_std_WoE,B_30_first_WoE,B_30_last_WoE,B_38_mean_WoE,B_38_std_WoE,B_38_first_WoE,B_38_last_WoE,D_114_mean_WoE,D_114_std_WoE,D_114_first_WoE,D_114_last_WoE,D_116_mean_WoE,D_116_std_WoE,D_116_first_WoE,D_116_last_WoE,D_117_mean_WoE,D_117_std_WoE,D_117_first_WoE,D_117_last_WoE,D_120_mean_WoE,D_120_std_WoE,D_120_first_WoE,D_120_last_WoE,D_126_mean_WoE,D_126_std_WoE,D_126_first_WoE,D_126_last_WoE,D_63_mean_WoE,D_63_std_WoE,D_63_first_WoE,D_63_last_WoE,D_64_mean_WoE,D_64_std_WoE,D_64_first_WoE,D_64_last_WoE,D_66_mean_WoE,D_66_std_WoE,D_66_first_WoE,D_66_last_WoE,D_68_mean_WoE,D_68_std_WoE,D_68_first_WoE,D_68_last_WoE
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,-0.390137,0.000000,-0.390137,-0.390137,-1.404297,0.370117,-0.751953,-1.590820,0.638672,0.203613,0.853027,0.466797,0.359131,0.468506,0.853027,-0.036072,0.448730,0.383545,0.853027,0.125366,0.927734,0.070923,0.853027,0.987793,0.302734,0.000000,0.302734,0.302734,-0.483154,0.0,-0.483154,-0.483154,0.578613,0.147217,0.733887,0.454346,0.042725,0.0,0.042725,0.042725,0.088257,0.635254,0.757812,-0.447510
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,-0.390137,0.000000,-0.390137,-0.390137,-1.365234,0.813477,-1.590820,-1.590820,-0.416260,0.000000,-0.416260,-0.416260,-0.036072,0.000000,-0.036072,-0.036072,0.106628,0.000000,0.106628,0.106628,-0.205200,0.000000,-0.205200,-0.205200,-0.089111,0.000000,-0.089111,-0.089111,0.072083,0.0,0.072083,0.072083,-0.487549,0.000000,-0.487549,-0.487549,0.042725,0.0,0.042725,0.042725,-0.447510,0.000000,-0.447510,-0.447510
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,-0.390137,0.000000,-0.390137,-0.390137,-1.590820,0.000000,-1.590820,-1.590820,0.466797,0.000000,0.466797,0.466797,-0.036072,0.000000,-0.036072,-0.036072,0.122437,0.007019,0.125366,0.106628,-0.205200,0.000000,-0.205200,-0.205200,0.181152,0.231079,-0.224243,0.302734,-0.483154,0.0,-0.483154,-0.483154,-0.270020,0.413086,-0.487549,0.454346,-0.389404,0.0,-0.389404,-0.389404,-0.239014,0.396240,-0.447510,0.456055
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.075256,0.884277,1.625977,-0.390137,0.388916,0.000000,0.388916,0.388916,0.466797,0.000000,0.466797,0.466797,-0.036072,0.000000,-0.036072,-0.036072,-0.578613,0.332031,0.125366,-0.456299,0.345459,0.619141,-0.205200,-0.205200,-0.089111,0.000000,-0.089111,-0.089111,0.212402,0.0,0.212402,0.212402,0.327637,0.000000,0.327637,0.327637,0.042725,0.0,0.042725,0.042725,0.068970,0.000000,0.068970,0.068970
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,1.161133,0.884277,-0.390137,1.625977,0.808594,0.557129,1.341797,1.571289,0.466797,0.000000,0.466797,0.466797,-0.036072,0.000000,-0.036072,-0.036072,0.125366,0.000000,0.125366,0.125366,-0.021698,0.447998,-0.205200,0.987793,0.302734,0.000000,0.302734,0.302734,0.072083,0.0,0.072083,0.072083,-0.173950,0.412842,-0.487549,0.327637,0.042725,0.0,0.042725,0.042725,-0.248901,0.261475,-0.447510,0.068970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,-0.390137,0.000000,-0.390137,-0.390137,-0.751953,0.000000,-0.751953,-0.751953,-0.416260,0.000000,-0.416260,-0.416260,-0.036072,0.000000,-0.036072,-0.036072,0.106628,0.000000,0.106628,0.106628,-0.205200,0.000000,-0.205200,-0.205200,-0.089111,0.000000,-0.089111,-0.089111,0.072083,0.0,0.072083,0.072083,-0.487549,0.000000,-0.487549,-0.487549,0.042725,0.0,0.042725,0.042725,-0.447510,0.000000,-0.447510,-0.447510
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,-0.390137,0.000000,-0.390137,-0.390137,0.462158,0.264404,0.388916,0.388916,-0.416260,0.000000,-0.416260,-0.416260,-0.036072,0.000000,-0.036072,-0.036072,0.106628,0.000000,0.106628,0.106628,0.804199,0.447998,0.987793,-0.205200,-0.089111,0.000000,-0.089111,-0.089111,0.072083,0.0,0.072083,0.072083,-0.487549,0.000000,-0.487549,-0.487549,0.042725,0.0,0.042725,0.042725,-0.2

In [38]:
#WoE Cat for reverth1
dataset = 'train'
data_df,label_df = data_process_loadfiles(dataset)
data_df = data_process_baseUpdate(data_df)
data_df = data_df[data_df['period_reverth']==1]
for item in cat_list:
        NA_count = data_df[item].isna().sum()
        if NA_count >0:
            #NA_dict[item+str('-in_cat')] = NA_count
            #print(item)
            data_df[item] = data_df[item].cat.add_categories("NA").fillna("NA")


rare_encoder = RareLabelEncoder(tol=0.01, n_categories=2, variables=cat_list,replace_with='Rare')

# set up a weight of evidence encoder
data_df_review = rare_encoder.fit_transform(data_df[cat_list])
woe_encoder = WoEEncoder(variables=cat_list)
woe_encoder.fit(data_df_review, data_df['target'])
train_t = woe_encoder.transform(data_df_review)
train_t[['customer_ID','S_2_dt']] = data_df[['customer_ID','S_2_dt']]

temp_df = dataP_basestat(train_t,cat_list)
temp_df.columns = [col + '_WoEonlylast' if col!='customer_ID' else col for col in temp_df.columns ]
temp_df.to_feather(Huge_data_path + 'cat__WoEonlylast_baseset' +'_'+dataset+'.ftr')

#WoE Cat test
dataset = 'test'
test_df,label_df = data_process_loadfiles(dataset)
test_df = data_process_baseUpdate(test_df)
test_df = test_df[test_df['period_reverth']==1]

for item in cat_list:
        NA_count = test_df[item].isna().sum()
        if NA_count >0:
            #NA_dict[item+str('-in_cat')] = NA_count
            #print(item)
            test_df[item] = test_df[item].cat.add_categories("NA").fillna("NA")
test_df[cat_list] = rare_encoder.transform(test_df[cat_list])
test_t = woe_encoder.transform(test_df[cat_list])
test_t[['customer_ID','S_2_dt']] = test_df[['customer_ID','S_2_dt']]
temp_df = dataP_basestat(test_t,cat_list)
temp_df.columns = [col + '_WoEonlylast' if col!='customer_ID' else col for col in temp_df.columns ]
temp_df.to_feather(Huge_data_path + 'cat__WoEonlylast_baseset' +'_'+dataset+'.ftr')
temp_df


ValueError: Length mismatch: Expected axis has 45 elements, new values have 44 elements

In [7]:
#isna_df
for dataset in ['train','test']:
    data_df,label_df = data_process_loadfiles(dataset)
    data_df = data_process_baseUpdate(data_df)

    isna_df = data_df[NA_review_list+['customer_ID']].isna()#.groupby('customer_ID').sum()
    isna_df.columns = [col + '_na' for col in isna_df.columns]
    isna_df['customer_ID'] = data_df['customer_ID']
    isna_df = isna_df.groupby('customer_ID').mean()
    isna_df.columns = [x + '_mean' for x in isna_df.columns]
    isna_df = isna_df.astype('float16')
    isna_df.reset_index().to_feather(Huge_data_path + 'NAset' +'_'+dataset+'.ftr')

In [6]:
#PCA tranform save

data_df = data_process_fillNA(data_df,num_list_inModel+num_list_XXXize)
pca = PCA(0.98)
#result = pca.fit_transform(data_df['num_list']) # Assume X is having more than 2 dimensions 
pca.fit(data_df[num_list])
pk.dump(pca, open("Model\pca.pkl","wb"))



In [6]:
#apply same train PCA transform to train
# later reload the pickle file
dataset = 'train'
data_df,label_df = data_process_loadfiles(dataset)
data_df = data_process_baseUpdate(data_df)
data_df, num_list_XXXize = data_process_itemXXXize(data_df)
num_list = num_list_inModel+num_list_XXXize
data_df = data_process_fillNA(data_df,num_list_inModel+num_list_XXXize)
pca = pk.load(open("Model\pca.pkl",'rb'))
pca_feature = pca.transform(data_df[num_list])
r,l = pca_feature.shape
col_list = list()
for i in range(l):
    col_list.append("PCA_"+str(i))

principalDf = pd.DataFrame(data = pca_feature, columns = col_list)
principalDf = pd.concat([data_df[['customer_ID','period_reverth']],principalDf],axis=1)
principalDf = principalDf[principalDf['period_reverth']==1]
principalDf = principalDf.reset_index(drop = True).drop('period_reverth',axis=1)
principalDf.to_feather(Huge_data_path + 'PCAset' +'_'+dataset+'.ftr')
principalDf

,customer_ID,PCA_0,PCA_1,PCA_2,PCA_3,PCA_4,PCA_5,PCA_6,PCA_7,PCA_8,PCA_9,PCA_10,PCA_11
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,-0.150713,0.622711,-0.247959,-0.101699,0.108856,-0.102978,-0.200473,0.325705,-1.090636,0.778227,-0.225008,0.008852
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,-0.148189,-1.750171,-0.251786,-0.184466,0.078206,-0.129943,-0.212394,0.359072,-1.152450,0.417159,-0.529508,0.098692
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,-0.147270,-0.318082,-0.254318,-0.177412,0.073355,-0.142676,-0.233511,0.381788,-1.375655,-0.761476,-0.031644,0.110587
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,-0.151226,-0.586620,-0.248927,-0.172215,0.193116,-0.117701,-0.199933,0.312485,-0.935069,0.473011,-0.433417,0.114172
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,-0.148609,0.751675,-0.239978,-0.081663,-0.108950,-0.128260,-0.157534,0.006161,-0.606536,-0.944343,0.256155,-0.059627
...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,-0.152721,0.379246,-0.251591,-0.194483,0.225414,-0.081010,-0.207248,0.474097,-0.864602,0.937186,-0.120155,0.138814
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,-0.148143,-1.050081,-0.231725,0.008817,-0.228531,-0.100164,-0.073160,-0.412570,0.704089,-1.152519,0.584647,-0.268646
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,-0.155250,-1.405450,-0.253448,-0.152286,0.082693,-0.116606,-0.195442,0.301387,-0.867284,0.558062,-0.484202,0.028686
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,-0.146574,-0.835660,-0.235518,0.133090,-0.205536,-0.105786,-0.112386,-0.347172,0.703861,-0.290827,-0.202011,-0.134471


In [7]:
#apply same train PCA transform to test
dataset = 'test'
data_df,label_df = data_process_loadfiles(dataset)
data_df = data_process_baseUpdate(data_df)
data_df, num_list_XXXize = data_process_itemXXXize(data_df)
num_list = num_list_inModel+num_list_XXXize
data_df = data_process_fillNA(data_df,num_list_inModel+num_list_XXXize)

pca = pk.load(open("Model\pca.pkl",'rb'))
pca_feature = pca.transform(data_df[num_list])
r,l = pca_feature.shape
col_list = list()
for i in range(l):
    col_list.append("PCA_"+str(i))

principalDf = pd.DataFrame(data = pca_feature, columns = col_list)
principalDf = pd.concat([data_df[['customer_ID','period_reverth']],principalDf],axis=1)
principalDf = principalDf[principalDf['period_reverth']==1]
principalDf = principalDf.reset_index(drop = True).drop('period_reverth',axis=1)
principalDf.to_feather(Huge_data_path + 'PCAset' +'_'+dataset+'.ftr')
principalDf

,customer_ID,PCA_0,PCA_1,PCA_2,PCA_3,PCA_4,PCA_5,PCA_6,PCA_7,PCA_8,PCA_9,PCA_10,PCA_11
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,-0.146801,-2.925544,-0.241468,0.174150,-0.192912,-0.106575,-0.184261,-0.035194,-0.262135,0.366549,-0.351996,0.027789
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,-0.154237,-2.090897,-0.253331,-0.199362,0.081695,-0.121033,-0.213134,0.353879,-1.022692,0.551720,-0.517780,0.119616
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,-0.146463,-2.023863,-0.251346,-0.103026,-0.104590,-0.127170,-0.162367,-0.049502,-0.335327,-0.991514,0.324697,-0.049572
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,-0.151479,-1.371276,-0.234271,0.034478,-0.214370,-0.029226,-0.022557,-0.577018,1.607656,0.187152,0.276174,-0.272023
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,-0.144960,-1.880092,-0.228800,-0.111624,-0.231015,-0.072523,0.021375,-0.719237,2.048291,-0.234512,0.144863,-0.301581
...,...,...,...,...,...,...,...,...,...,...,...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,-0.148099,-1.805852,-0.246790,-0.010485,-0.177263,-0.122246,-0.172895,0.054896,-0.698370,0.000577,-0.342302,-0.028020
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,-0.148244,-2.286698,-0.230138,0.171315,-0.202645,-0.083525,-0.029871,-0.510562,1.396491,-0.172135,0.004129,1.155288
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,-0.144899,-2.675297,-0.236860,0.028029,-0.192984,-0.092639,-0.062934,-0.335606,0.695190,-0.071235,-0.132777,-0.140370
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,-0.154213,-2.032841,-0.232362,0.215008,-0.223359,-0.096042,-0.027449,-0.613363,1.514574,-0.563478,0.070937,-0.308781


In [11]:
data_df = data_df[data_df['period_reverth']==1]
principalDf = data_df.set[col_list]
principalDf

,B_1,B_10,B_11,B_12,B_13,B_14,B_15,B_16,B_17,B_18,B_19,B_2,B_20,B_21,B_22,B_23,B_24,B_25,B_26,B_27,B_28,B_29,B_3,B_30,B_31,B_32,B_33,B_36,B_37,B_38,B_39,B_4,B_40,B_41,B_42,B_5,B_6,B_7,B_8,B_9,D_102,D_103,D_104,D_105,D_106,D_107,D_108,D_109,D_110,D_111,D_112,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,D_130,D_131,D_132,D_133,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,D_39,D_41,D_42,D_43,D_44,D_45,D_46,D_47,D_48,D_49,D_50,D_51,D_52,D_53,D_54,D_55,D_56,D_58,D_59,D_60,D_61,D_62,D_63,D_64,D_65,D_66,D_68,D_69,D_70,D_71,D_72,D_73,D_74,D_75,D_76,D_77,D_78,D_79,D_80,D_81,D_82,D_83,D_84,D_86,D_87,D_88,D_89,D_91,D_92,D_93,D_94,D_96,P_2,P_3,P_4,R_1,R_10,R_11,R_12,R_13,R_14,R_15,R_16,R_17,R_18,R_19,R_2,R_20,R_21,R_22,R_23,R_24,R_25,R_26,R_27,R_28,R_3,R_4,R_5,R_6,R_7,R_8,R_9,S_11,S_12,S_13,S_15,S_16,S_17,S_18,S_19,S_2,S_20,S_22,S_23,S_24,S_25,S_26,S_27,S_3,S_5,S_6,S_7,S_8,S_9,customer_ID,period_count,target,S_2_dt,S_2_dt_d,S_2_dt_m,S_2_dt_y,S_2_dt_wd,S_2_dt_sm,S_2_dt_m_in3,period_th,period_reverth,S_2_yearto2018Mar31,D_55_cen,D_56_cen,D_60_cen,D_61_cen,R_7_cen,S_7_cen,D_73_std,D_76_std,R_9_std,S_11_std,S_5_std,S_8_std,S_9_std,D_59_Q50cen,PAC_0,PAC_1,PAC_2,PAC_3,PAC_4,PAC_5,PAC_6,PAC_7,PAC_8,PAC_9,PAC_10,PAC_11
0,0.008728,0.096191,0.002768,0.148315,0.118103,0.018387,0.016357,0.007664,0.666504,0.652832,0.008522,1.006836,0.004730,0.002644,0.004807,0.050873,0.004326,0.007729,0.000272,0.002310,0.084656,NaN,0.004707,0.0,1.0,0.006626,1.000977,0.009972,0.004570,2.0,NaN,0.080994,0.210083,0.006805,NaN,0.170654,0.063904,0.059418,0.006466,0.008209,0.766602,1.008789,1.004883,0.893555,NaN,0.669922,NaN,0.004326,NaN,NaN,1.007812,0.007874,1.0,0.238281,0.0,4.0,0.232178,0.236206,0.0,0.702148,0.434326,0.003057,0.686523,0.008743,1.0,1.002930,1.007812,1.000000,0.002052,0.005970,NaN,0.004345,NaN,NaN,NaN,NaN,NaN,0.002426,0.003706,0.003819,NaN,0.000569,0.000610,0.002674,0.001734,0.008774,NaN,0.142456,0.000630,0.708984,0.358643,0.525391,0.255615,NaN,0.148682,1.335938,0.207275,NaN,1.001953,0.354492,0.151978,0.158569,0.063660,0.199585,0.308350,0.091064,CR,O,0.007126,NaN,6.0,0.009010,0.008339,0.119385,0.004803,NaN,0.080444,0.069092,NaN,0.419434,0.001575,0.004238,0.004059,0.003532,0.506836,0.007042,0.000830,0.007084,0.0,NaN,0.002665,1.503906,1.005859,0.003569,0.008873,0.004951,0.938477,0.736328,0.007553,0.009224,0.007122,0.002457,1.009766,0.003820,0.000438,0.006451,0.005054,0.000198,0.008911,0.005177,0.006203,0.007782,0.002449,0.007481,0.006893,0.003948,0.003647,NaN,1.008789,0.001534,0.001423,0.008301,0.001882,0.008362,0.007561,0.001434,NaN,0.401611,0.271973,0.515137,0.108276,0.002272,0.008034,0.005722,0.002537,2017-03-09,0.009705,0.894043,0.135620,0.911133,0.974609,0.001244,0.676758,0.124023,0.023376,0.008324,0.161377,0.922852,0.065735,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,13,0,2017-03-09,9,3,2017,3,0,0,1.0,13.0,-1.059570,0.068176,-0.057068,-0.173828,-0.096863,0.001314,-0.055115,-3.205078,-0.219360,-1.143555,-1.856445,-0.484619,-0.172485,-0.330078,-0.216064,-0.146728,-0.242282,-0.248156,0.008258,-0.121960,-0.097516,-0.172491,0.205462,-0.966406,0.862529,-0.105750,-0.034250
1,0.004925,0.099792,0.002748,0.143555,0.118713,0.013039,0.017685,0.007149,0.680176,0.646973,0.002237,1.000977,0.003880,0.004192,0.001283,0.040466,0.004204,0.001864,0.000978,0.001327,0.081848,NaN,0.002714,0.0,1.0,0.001854,1.006836,0.003922,0.004654,2.0,NaN,0.069397,0.184082,0.004406,NaN,0.113220,0.065247,0.057739,0.001614,0.008369,0.786133,1.000000,1.003906,0.906738,NaN,0.668457,NaN,0.008705,NaN,NaN,1.007812,0.003445,1.0,0.247192,0.0,4.0,0.243530,0.241943,0.0,0.707031,0.430420,0.001306,0.686523,0.000755,1.0,1.008789,1.003906,1.008789,0.001034,0.004837,NaN,0.007496,NaN,NaN,NaN,NaN,NaN,0.003956,0.003166,0.005032,NaN,0.009575,0.005493,0.009216,0.005775,0.000798,NaN,0.146362,0.002525,0.712891,0.353516,0.521484,0.223389,NaN,0.149780,1.339844,0.202759,NaN,1.008789,0.326660,0.156250,0.148438,0.065491,0.151367,0.

In [6]:
dataset = 'test'
data_df,label_df = data_process_loadfiles(dataset)
data_df = data_process_baseUpdate(data_df)
data_df, num_list_XXXize = data_process_itemXXXize(data_df)

In [7]:
#baseset

temp_df = dataP_basestat(data_df,num_list_inModel+num_list_XXXize)
temp_df.to_feather(Huge_data_path + 'baseset' +'_'+dataset+'.ftr')


AttributeError: 'str' object has no attribute 'sort_values'

In [7]:
#baseset_withdroped
for dataset in ['train','test'] :
    data_df,label_df = data_process_loadfiles(dataset)
    data_df = data_process_baseUpdate(data_df)
    data_df, num_list_XXXize = data_process_itemXXXize(data_df)
    temp_df = dataP_basestat(data_df,num_list+num_list_XXXize)
    temp_df.to_feather(Huge_data_path + 'baseset_withdroped' +'_'+dataset+'.ftr')

In [12]:
#Data Order gen
#here num_list = num_list+num_list_XXXize
data_df = data_df.sort_values('S_2_dt')# early to latest #,ascending = False) #latest to early 
data_df[num_list] = data_df[num_list].astype('float32')
data_last = data_df.groupby("customer_ID")[num_list].last() #lag_org
data_frist = data_df.groupby("customer_ID")[num_list].first() #for NA fill
order_agg = pd.DataFrame(index = data_last.index)
#for k in range(1,7):
for k in [1,3,6,'f']:
    print('lag'+str(k), end = "\r", flush=True)
    data_n = pd.DataFrame(index=data_last.index)
    if k == 'f':
        data_n = data_n.join(data_frist,on='customer_ID')
    else:
        data_n = data_n.join(data_df.groupby('customer_ID').nth(-k-1)[num_list],on='customer_ID')
        data_n[data_n.isnull()] = data_frist
    col_list = data_n.columns

    data_n[num_list] = data_last[num_list] - data_n[num_list]
    data_n.columns =  [str(col) + '_lag'+str(k)+'_sub' for col in col_list]
    order_agg = order_agg.join(data_n,on='customer_ID')

    data_n.columns =  col_list
    data_n[num_list] = data_n[num_list]/data_last[num_list]
    data_n.columns =  [str(col) + '_lag'+str(k)+'_prop' for col in col_list]
    order_agg = order_agg.join(data_n,on='customer_ID')
order_agg = order_agg.astype('float16').reset_index()

order_agg.to_feather(Huge_data_path + 'single_lagset' +'_'+dataset+'.ftr')
order_agg

customer_ID  B_1_lag1_sub  \
0       00000469ba478561f23a92a868bd366de6f6527a684c9a...     -0.006645   
1       00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...     -0.037415   
2       0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...     -0.005344   
3       00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...     -0.125366   
4       00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...      0.054199   
...                                                   ...           ...   
924616  ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...      0.004463   
924617  ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...      0.017090   
924618  ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...     -0.011597   
924619  ffffddef1fc3643ea179c93245b68dca0f36941cd83977...     -0.015961   
924620  fffffa7cf7e453e1acc6a1426475d5cb9400859f82ff61...     -0.009186   

        B_10_lag1_sub  B_11_lag1_sub  B_12_lag1_sub  B_13_lag1_sub  \
0           -0.003815       0.000622      -0.008820      -0.004517   
1           -0.004395      -0.038757       0.002045      -0.000580   
2           -0.014404      -0.001831      -0.005524       0.000061   
3            0.013687      -0.146606       0.007202       0.083374   
4           -0.003143       0.049316      -0.001648      -0.011459   
...               ...            ...            ...            ...   
924616       0.025925      -0.006229      -0.005005       0.000664   
924617       0.006073       0.009674       0.006500      -0.001938   
924618      -0.008087      -0.008049       0.004242      -0.002899   
924619       0.003113      -0.007401       0.002518      -0.007221   
924620       0.000977      -0.007065      -0.002190       0.000086   

        B_14_lag1_sub  B_15_lag1_sub  B_16_lag1_sub  B_17_lag1_sub  \
0           -0.011459      -0.006859       0.002089            NaN   
1           -0.039490       0.002659       0.089050            NaN   
2            0.002382      -0.001911      -0.001999            NaN   
3           -0.211670      -0.161377       0.000000      -0.257324   
4            0.010620      -0.002449       0.008789      -0.016113   
...               ...            ...            ...            ...   
924616       0.002827       0.006641      -0.002745            NaN   
924617       0.004395       0.006405      -0.008789       0.002930   
924618      -0.004868      -0.007339       0.000412            NaN   
924619      -0.003021       0.000510       0.000977      -0.076660   
924620       0.006493      -0.001011       0.003273            NaN   

        B_18_lag1_sub  B_19_lag1_sub  B_2_lag1_sub  B_20_lag1_sub  \
0           -0.000488      -0.005627      0.008789      -0.005318   
1            0.001953      -0.005905      0.007812      -0.001118   
2           -0.001953       0.000003     -0.000977      -0.002522   
3            0.005859       0.000977      0.129639       0.001953   
4            0.003052      -0.076172      0.005280      -0.004883   
...               ...            ...           ...            ...   
924616      -0.007324      -0.007812     -0.001465       0.008919   
924617      -0.052979      -0.001854     -0.000793       0.004639   
924618       0.004883      -0.000454     -0.004883       0.001770   
924619       0.074585       0.002640      0.030884       0.007812   
924620       0.005859      -0.006378     -0.006348      -0.002716   

        B_21_lag1_sub  B_22_lag1_sub  B_23_lag1_sub  B_24_lag1_sub  \
0            0.006565       0.003967       0.007446       0.000946   
1           -0.001354      -0.000067      -0.023041       0.001514   
2            0.000015      -0.001785      -0.012512       0.000183   
3            0.001780      -0.506836      -0.088989      -0.004707   
4            0.004601       0.499023       0.028687       0.007221   
...               ...            ...            ...            ...   
924616       0.002569       0.008598      -0.010620      -0.004044   
924617       0.745117      -0.002941       0.066650      -0.004799   
9246

In [13]:
#dataP_avgstat
#here num_list = num_list+num_list_XXXize
output_df = pd.DataFrame(index = data_df['customer_ID'].unique()).rename_axis(index = 'customer_ID')
for j in [3,6,]:
    num_agg = data_df.sort_values('S_2_dt').groupby("customer_ID")[num_list].nth(range(j)).groupby("customer_ID").mean()
    std_agg = data_df.sort_values('S_2_dt').groupby("customer_ID")[num_list].nth(range(j)).groupby("customer_ID").std()
    data_last = data_df.groupby("customer_ID")[num_list].last() #lag_org
    diff_agg = pd.DataFrame()
    diff_agg = data_last - num_agg

    num_agg.columns = [str(col) + '_f'+str(j)+'avg' for col in num_agg.columns]
    diff_agg.columns = [str(col) + '_lag_diff_f'+str(j)+'avg' for col in diff_agg.columns]
    std_agg.columns = [str(col) + '_f'+str(j)+'std' for col in std_agg.columns]

    output_df = output_df.join(num_agg,on='customer_ID')
    output_df = output_df.join(diff_agg,on='customer_ID')
    output_df = output_df.join(std_agg,on='customer_ID')

for j in [-3,-6,]:
    num_agg = data_df.sort_values('S_2_dt').groupby("customer_ID")[num_list].nth(range(-1,j-1,-1)).groupby("customer_ID").mean()
    std_agg = data_df.sort_values('S_2_dt').groupby("customer_ID")[num_list].nth(range(-1,j-1,-1)).groupby("customer_ID").std()
    data_last = data_df.groupby("customer_ID")[num_list].last() #lag_org
    diff_agg = pd.DataFrame()
    diff_agg = data_last - num_agg

    num_agg.columns = [str(col) + '_f'+str(j)+'avg' for col in num_agg.columns]
    diff_agg.columns = [str(col) + '_lag_diff_f'+str(j)+'avg' for col in diff_agg.columns]
    std_agg.columns = [str(col) + '_f'+str(j)+'std' for col in std_agg.columns]

    output_df = output_df.join(num_agg,on='customer_ID')
    output_df = output_df.join(diff_agg,on='customer_ID')
    output_df = output_df.join(std_agg,on='customer_ID')
output_df = output_df.reset_index()

output_df.to_feather(Huge_data_path + 'avgstatset' +'_'+dataset+'.ftr')
output_df

customer_ID  B_1_f3avg  \
0       af9da794a3ec613f9db1321559ffffcf6d5e0324b6f995...   0.703451   
1       615acaa0b35859ab76b13b3d5156acc6a208b8ec7b3b57...   1.041341   
2       ce0e53cf2a7a180578e81a5e2446f8aac58e04a70d3f97...   0.044881   
3       fcc9e53e0d05e55eb177e7ac5c9712c32671749013b657...   0.035655   
4       50f5d28c9700f06722794aae605a1d784a1702db845b8d...   0.022230   
...                                                   ...        ...   
924616  4817d9b6b43b70a9ba2e48ccb3aad12822461ec010095e...   0.003536   
924617  bce77041f86fcab6607ff0a1e4d80d7f31300400b073d6...   0.017334   
924618  db791e0a03452d219a9b312eb3a540640f4136426b047c...   0.142944   
924619  aa049a3092f20824cd2ae173ffaf233b9b4d86cb2cd16a...   0.099426   
924620  d4dbd14dc90bf4aff7325b78fee23e3fd2540fb3b437bd...   0.006046   

        B_10_f3avg  B_11_f3avg  B_12_f3avg  B_13_f3avg  B_14_f3avg  \
0         0.034403    0.706055    0.023575    0.175822    0.308105   
1         0.034968    1.020182    0.010951    0.016505    0.236491   
2         0.264872    0.032447    0.062724    0.068858    0.054316   
3         0.041626    0.023834    0.133993    0.094198    0.089050   
4         0.092194    0.010286    0.009961    0.011103    0.008934   
...            ...         ...         ...         ...         ...   
924616    0.244995    0.007416    0.005180    0.001529    0.001892   
924617    0.359131    0.015358    0.182617    0.445312    0.058594   
924618    0.020218    0.094177    0.009209    0.023972    0.023773   
924619    0.069580    0.063599    0.011635    0.057648    0.020538   
924620    0.245361    0.007504    0.004986    0.009933    0.003078   

        B_15_f3avg  B_16_f3avg  B_17_f3avg  B_18_f3avg  B_19_f3avg  B_2_f3avg  \
0         0.008904    0.782064    0.965820    0.173625    0.325439   0.083374   
1         0.006588    1.005208    0.987467    0.149984    0.973633   0.029282   
2         0.005847    0.086365    0.005214    0.769043    0.006196   1.004883   
3         0.005501    0.060909         NaN    0.549805    0.003723   1.004557   
4         0.003562    0.058509         NaN    0.623372    0.005868   0.879557   
...            ...         ...         ...         ...         ...        ...   
924616    0.003119    0.009537         NaN    1.007812    0.005203   0.815918   
924617    0.068298    0.008980         NaN    0.654297    0.003202   1.001953   
924618    0.004395    0.002752         NaN    0.689941    0.005169   0.813965   
924619    0.002409    0.091248         NaN    0.342773    0.002998   0.005695   
924620    0.006165    0.001777         NaN    1.003906    0.009109   0.813477   

        B_20_f3avg  B_21_f3avg  B_22_f3avg  B_23_f3avg  B_24_f3avg  \
0         0.358317    0.005847    0.172639    0.371094    0.004968   
1         1.005859    0.003553    0.504883    0.251994    0.003043   
2         0.027672    0.004288    0.009186    0.051188    0.005044   
3         0.004659    0.004279    0.003277    0.149577    0.005888   
4         0.005323    0.005457    0.003792    0.063690    0.003031   
...            ...         ...         ...         ...         ...   
924616    0.005203    0.001575    0.003542    0.008430    0.004398   
924617    0.002367    0.004822    0.003437    0.032196    1.115234   
924618    0.005096    0.004299    0.008789    0.292236    0.008705   
924619    0.066406    0.000654    0.008705    0.033508    0.009369   
924620    0.002918    0.005257    0.001843    0.007828    0.006573   

        B_25_f3avg  B_26_f3avg  B_27_f3avg  B_28_f3avg  B_3_f3avg  B_31_f3avg  \
0         0.546712    0.003931    0.006802    0.160970   0.249552         1.0   
1         0.416341    0.004105    0.006856    0.284505   0.779785         1.0   
2         0.084269    0.007463    0.005234    0.047719   0.007981         1.0   
3         0.148682    0.002127    0.003806    0.236979   0.004056         1.0   
4         0.010450    0.004433    0.004111    0.084559   0.005163         1.0   
...            ...         ...         .

In [14]:

#slope >2hours for train data......#37

review_T = pd.DataFrame()
for i, item in enumerate( Slope_items):
    for col in data_df.columns:
        if item == col or item+'_' in col:
            review_T[col+'_slope'] = data_df[['customer_ID','S_2_yearto2018Mar31',col]].groupby('customer_ID').apply(lambda x: linregress(x['S_2_yearto2018Mar31'],x[col])[0])
            print('Slope-'+str(i)+':'+ col, end = "\r", flush=True)

review_T.reset_index().to_feather(Huge_data_path + 'slopeset' +'_'+dataset+'.ftr')
review_T

KeyboardInterrupt: 

In [8]:
dataset = 'train'
data_df,label_df = data_process_loadfiles(dataset)
data_df = data_process_baseUpdate(data_df)
for item in cat_list:
        NA_count = data_df[item].isna().sum()
        if NA_count >0:
            #NA_dict[item+str('-in_cat')] = NA_count
            #print(item)
            data_df[item] = data_df[item].cat.add_categories("NA").fillna("NA")


rare_encoder = RareLabelEncoder(tol=0.01, n_categories=2, variables=cat_list,replace_with='Rare')

# set up a weight of evidence encoder
data_df_review = rare_encoder.fit_transform(data_df[cat_list])
woe_encoder = WoEEncoder(variables=cat_list)
woe_encoder.fit(data_df_review, data_df['target'])
train_t = woe_encoder.transform(data_df_review)
train_t[['customer_ID','S_2_dt','S_2_yearto2018Mar31']] = data_df[['customer_ID','S_2_dt','S_2_yearto2018Mar31']]


AttributeError: 'float' object has no attribute 'shape'

In [13]:
pd.read_feather(Huge_data_path+'package/cat_WoE_baseset_train.ftr')

,customer_ID,B_30_mean_WoE,B_30_std_WoE,B_30_first_WoE,B_30_last_WoE,B_38_mean_WoE,B_38_std_WoE,B_38_first_WoE,B_38_last_WoE,D_114_mean_WoE,D_114_std_WoE,D_114_first_WoE,D_114_last_WoE,D_116_mean_WoE,D_116_std_WoE,D_116_first_WoE,D_116_last_WoE,D_117_mean_WoE,D_117_std_WoE,D_117_first_WoE,D_117_last_WoE,D_120_mean_WoE,D_120_std_WoE,D_120_first_WoE,D_120_last_WoE,D_126_mean_WoE,D_126_std_WoE,D_126_first_WoE,D_126_last_WoE,D_63_mean_WoE,D_63_std_WoE,D_63_first_WoE,D_63_last_WoE,D_64_mean_WoE,D_64_std_WoE,D_64_first_WoE,D_64_last_WoE,D_66_mean_WoE,D_66_std_WoE,D_66_first_WoE,D_66_last_WoE,D_68_mean_WoE,D_68_std_WoE,D_68_first_WoE,D_68_last_WoE
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,-0.390137,0.000000,-0.390137,-0.390137,-1.590820,0.000000,-1.590820,-1.590820,-0.416260,0.000000,-0.416260,-0.416260,-0.036072,0.0,-0.036072,-0.036072,-0.456299,0.000000,-0.456299,-0.456299,-0.205200,0.000000,-0.2052,-0.205200,-0.089111,0.0,-0.089111,-0.089111,-0.483154,0.0,-0.483154,-0.483154,-0.487549,0.000000,-0.487549,-0.487549,0.042725,0.0,0.042725,0.042725,-0.447510,0.000000,-0.447510,-0.447510
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,-0.390137,0.000000,-0.390137,-0.390137,-1.590820,0.000000,-1.590820,-1.590820,-0.416260,0.000000,-0.416260,-0.416260,-0.036072,0.0,-0.036072,-0.036072,0.125366,0.000000,0.125366,0.125366,0.253662,0.604004,-0.2052,-0.205200,-0.089111,0.0,-0.089111,-0.089111,0.072083,0.0,0.072083,0.072083,-0.487549,0.000000,-0.487549,-0.487549,0.042725,0.0,0.042725,0.042725,-0.447510,0.000000,-0.447510,-0.447510
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,-0.390137,0.000000,-0.390137,-0.390137,-0.751953,0.000000,-0.751953,-0.751953,-0.212524,0.387207,0.466797,-0.416260,-0.036072,0.0,-0.036072,-0.036072,0.125366,0.000000,0.125366,0.125366,-0.205200,0.000000,-0.2052,-0.205200,-0.089111,0.0,-0.089111,-0.089111,0.072083,0.0,0.072083,0.072083,0.327637,0.000000,0.327637,0.327637,0.042725,0.0,0.042725,0.042725,-0.447510,0.000000,-0.447510,-0.447510
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,-0.390137,0.000000,-0.390137,-0.390137,-1.590820,0.000000,-1.590820,-1.590820,-0.416260,0.000000,-0.416260,-0.416260,-0.036072,0.0,-0.036072,-0.036072,-0.557129,0.389160,0.125366,-0.761719,-0.205200,0.000000,-0.2052,-0.205200,-0.089111,0.0,-0.089111,-0.089111,0.072083,0.0,0.072083,0.072083,-0.487549,0.000000,-0.487549,-0.487549,0.042725,0.0,0.042725,0.042725,0.428711,0.390625,0.668457,0.570312
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,-0.390137,0.000000,-0.390137,-0.390137,-1.138672,0.435547,-1.590820,-0.751953,-0.416260,0.000000,-0.416260,-0.416260,-0.036072,0.0,-0.036072,-0.036072,-0.456299,0.000000,-0.456299,-0.456299,-0.205200,0.000000,-0.2052,-0.205200,-0.089111,0.0,-0.089111,-0.089111,0.072083,0.0,0.072083,0.072083,-0.487549,0.000000,-0.487549,-0.487549,-0.389404,0.0,-0.389404,-0.389404,-0.447510,0.000000,-0.447510,-0.447510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,-0.390137,0.000000,-0.390137,-0.390137,-1.590820,0.000000,-1.590820,-1.590820,0.466797,0.000000,0.466797,0.466797,-0.036072,0.0,-0.036072,-0.036072,-0.282959,0.270264,-0.456299,0.106628,0.437012,0.619141,-0.2052,0.987793,-0.089111,0.0,-0.089111,-0.089111,0.072083,0.0,0.072083,0.072083,0.454346,0.000000,0.454346,0.454346,-0.389404,0.0,-0.389404,-0.389404,-0.447510,0.000000,-0.447510,-0.447510
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,1.161133,0.884277,1.625977,-0.390137,1.043945,0.196899,0.388916,1.098633,-0.416260,0.000000,-0.416260,-0.416260,-0.036072,0.0,-0.036072,-0.036072,0.236206,0.170654,0.443604,0.443604,-0.205200,0.000000,-0.2052,-0.205200,-0.089111,0.0,-0.089111,-0.089111,0.072083,0.0,0.072083,0.072083,0.327637,0.000000,0.327637,0.327637,0.042725,0.0,0.042725,0.042725,-0.447510,0.000000,-0.447510,-0.447510
458910,ffff9

In [15]:

review_T = pd.DataFrame()
for i, item in enumerate( cat_list):
    for col in train_t.columns:
        if item == col or item+'_' in col:
            review_T[col+'_Woeslope'] = train_t[['customer_ID','S_2_yearto2018Mar31',col]].groupby('customer_ID').apply(lambda x: linregress(x['S_2_yearto2018Mar31'],x[col])[0])
            print('_Woeslope-'+str(i)+':'+ col, end = "\r", flush=True)

review_T.reset_index().to_feather(Huge_data_path + 'Woe_slopeset' +'_'+dataset+'.ftr')
review_T

,B_30_Woeslope,B_38_Woeslope,D_114_Woeslope,D_116_Woeslope,D_117_Woeslope,D_120_Woeslope,D_126_Woeslope,D_63_Woeslope,D_64_Woeslope,D_66_Woeslope,D_68_Woeslope
customer_ID,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,-2.164667e-32,8.658668e-32,-2.164667e-32,-2.705834e-33,2.164667e-32,1.082333e-32,0.0,2.164667e-32,-2.164667e-32,0.0,0.000000
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,5.449007e-32,-2.179603e-31,5.449007e-32,6.811258e-33,2.724503e-32,2.002879e-01,0.0,-1.362252e-32,5.449007e-32,0.0,0.000000
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,-7.028269e-32,0.000000e+00,-8.714579e-01,-8.785336e-33,-3.514134e-32,3.514134e-32,0.0,1.757067e-32,7.028269e-32,0.0,0.000000
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-8.666231e-01,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.387078
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed,-9.379552e-33,1.155809e+00,-9.379552e-33,-1.172444e-33,9.379552e-33,4.689776e-33,0.0,2.344888e-33,-9.379552e-33,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
ffff41c8a52833b56430603969b9ca48d208e7c192c6a4081a6acc28cf4f8af7,4.760730e-32,-1.904292e-31,4.760730e-32,5.950912e-33,2.179317e-02,1.598385e+00,0.0,-1.190182e-32,0.000000e+00,0.0,0.000000
ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fdd3e5b57cfcbee30286,-1.169141e+00,2.710876e-01,4.730493e-33,5.913116e-34,2.631163e-01,-2.365246e-33,0.0,-1.182623e-33,-4.730493e-33,0.0,0.000000
ffff9984b999fccb2b6127635ed0736dda94e544e67e026eee4d20f680639ff6,-3.099447e-32,7.563281e-01,-3.099447e-32,-3.874309e-33,-1.549724e-32,1.549724e-32,0.0,7.748618e-33,0.000000e+00,0.0,0.000000


In [16]:
dataset = 'test'
data_df,label_df = data_process_loadfiles(dataset)
data_df = data_process_baseUpdate(data_df)
for item in cat_list:
        NA_count = data_df[item].isna().sum()
        if NA_count >0:
            #NA_dict[item+str('-in_cat')] = NA_count
            #print(item)
            data_df[item] = data_df[item].cat.add_categories("NA").fillna("NA")


#rare_encoder = RareLabelEncoder(tol=0.01, n_categories=2, variables=cat_list,replace_with='Rare')

# set up a weight of evidence encoder
data_df_review = rare_encoder.fit_transform(data_df[cat_list])
#woe_encoder = WoEEncoder(variables=cat_list)
#woe_encoder.fit(data_df_review, data_df['target'])
train_t = woe_encoder.transform(data_df_review)
train_t[['customer_ID','S_2_dt','S_2_yearto2018Mar31']] = data_df[['customer_ID','S_2_dt','S_2_yearto2018Mar31']]

review_T = pd.DataFrame()
for i, item in enumerate( cat_list):
    for col in train_t.columns:
        if item == col or item+'_' in col:
            review_T[col+'_Woeslope'] = train_t[['customer_ID','S_2_yearto2018Mar31',col]].groupby('customer_ID').apply(lambda x: linregress(x['S_2_yearto2018Mar31'],x[col])[0])
            print('_Woeslope-'+str(i)+':'+ col, end = "\r", flush=True)

review_T.reset_index().to_feather(Huge_data_path + 'Woe_slopeset' +'_'+dataset+'.ftr')
review_T

,B_30_slope,B_38_slope,D_114_slope,D_116_slope,D_117_slope,D_120_slope,D_126_slope,D_63_slope,D_64_slope,D_66_slope,D_68_slope
customer_ID,,,,,,,,,,,
00000469ba478561f23a92a868bd366de6f6527a684c9a2e78fb826dcac3b9b7,0.000000e+00,-1.189627e+00,-7.750894e-01,-1.783730e+00,-1.459938e+00,2.700477e-01,0.000000e+00,0.000000e+00,-5.604799e-01,0.0,-2.418400
00001bf2e77ff879fab36aa4fac689b9ba411dae63ae397d4263dafa1daedef5,5.189800e-33,6.596289e-01,5.189800e-33,6.487251e-34,0.000000e+00,-2.594900e-33,0.000000e+00,-1.297450e-33,5.189800e-33,0.0,0.000000
0000210045da4f81e5f122c6bde5c2a617d03eef67f82c5e400fc98e7bd43ce8,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.303182e-02,0.000000e+00,5.250715e-01,0.000000e+00,9.087107e-01,0.0,0.871761
00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976cf6e56734528702d694,-1.860802e+00,-4.495868e-32,-4.495868e-32,-5.619835e-33,-4.062785e-01,-6.770206e-01,0.000000e+00,2.247934e-32,4.495868e-32,0.0,0.000000
00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9a4693dd914fca22557,1.772976e+00,5.177158e-01,4.137812e-32,5.172265e-33,2.068906e-32,9.252068e-01,-4.137812e-32,-1.034453e-32,1.076770e+00,0.0,0.682181
...,...,...,...,...,...,...,...,...,...,...,...
ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c5d60460dba6dedc41e,-9.725780e-33,0.000000e+00,-9.725780e-33,-1.215722e-33,0.000000e+00,4.862890e-33,0.000000e+00,2.431445e-33,-9.725780e-33,0.0,0.000000
ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3a4f0ca3de613b0b2ad,0.000000e+00,-2.986545e-01,0.000000e+00,0.000000e+00,0.000000e+00,-8.652710e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.0,-0.654761
ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475cb095d2443a68030f1,-5.467958e-32,-5.467958e-32,-5.467958e-32,-6.834948e-33,0.000000e+00,-1.194154e+00,0.000000e+00,5.467958e-32,-5.467958e-32,0.0,0.000000


In [20]:
review_T.columns = ['B_30_Woeslope', 'B_38_Woeslope', 'D_114_Woeslope', 'D_116_Woeslope', 'D_117_Woeslope',
       'D_120_Woeslope', 'D_126_Woeslope', 'D_63_Woeslope', 'D_64_Woeslope', 'D_66_Woeslope',
       'D_68_Woeslope']
review_T.reset_index().to_feather(Huge_data_path + 'Woe_slopeset' +'_'+dataset+'.ftr')


run train slope here

In [1]:
#ok.....that combina the XXXize record
%run "10_Data into model format.ipynb"
dataset = 'train'
data_df,label_df = data_process_loadfiles(dataset)
data_df = data_process_baseUpdate(data_df)
data_df, num_list_XXXize = data_process_itemXXXize(data_df)
num_list = num_list_inModel+num_list_XXXize
#save data df - save feather need convert category

review_T = pd.DataFrame()
for i, item in enumerate( num_list_XXXize):
    for col in data_df.columns:
        if item == col or item+'_' in col:
            review_T[col+'_slope'] = data_df[['customer_ID','S_2_yearto2018Mar31',col]].groupby('customer_ID').apply(lambda x: linregress(x['S_2_yearto2018Mar31'],x[col])[0])
            print('Slope-'+str(i)+':'+ col, end = "\r", flush=True)

review_Train = pd.read_pickle(Huge_data_path + 'slope_pre_record' +'_'+'train'+'.plk')#.set_index('customer_ID')
review_Test= pd.read_feather(Huge_data_path + 'slopeset' +'_'+'test'+'.ftr').set_index('customer_ID')
review_Train = review_Train.join(review_T,on='customer_ID')
review_Train = review_Train[review_Test.columns]
review_Train.reset_index().to_feather(Huge_data_path + 'slopeset' +'_'+dataset+'.ftr')
review_T

function: load_DProcessed_df(file), amex_metric(y_true, y_pred), file_get(dataset,version,dataset_list,num_list_inModel,ignore_list)
list:all_cols,cat_list,num_list , varible: Huge_data_path


FileNotFoundError: [Errno 2] No such file or directory: '../huge data source/AE/slopeset_train.ftr'

In [27]:
# #logic!!
# [col for col in temp_df.columns if 
#     (col.startswith(tuple([str(col) + '_' for col in num_list_inModel])))and #logic that contain items
#     (all(s not in col for s in ignore_list)) #logic that calc ignore
#     ]

['B_10_lag1_sub',
 'B_11_lag1_sub',
 'B_12_lag1_sub',
 'B_13_lag1_sub',
 'B_14_lag1_sub',
 'B_15_lag1_sub',
 'B_16_lag1_sub',
 'B_17_lag1_sub',
 'B_18_lag1_sub',
 'B_19_lag1_sub',
 'B_2_lag1_sub',
 'B_20_lag1_sub',
 'B_21_lag1_sub',
 'B_22_lag1_sub',
 'B_23_lag1_sub',
 'B_24_lag1_sub',
 'B_25_lag1_sub',
 'B_26_lag1_sub',
 'B_27_lag1_sub',
 'B_28_lag1_sub',
 'B_3_lag1_sub',
 'B_31_lag1_sub',
 'B_32_lag1_sub',
 'B_33_lag1_sub',
 'B_36_lag1_sub',
 'B_37_lag1_sub',
 'B_4_lag1_sub',
 'B_40_lag1_sub',
 'B_41_lag1_sub',
 'B_5_lag1_sub',
 'B_6_lag1_sub',
 'B_7_lag1_sub',
 'B_8_lag1_sub',
 'B_9_lag1_sub',
 'D_102_lag1_sub',
 'D_103_lag1_sub',
 'D_104_lag1_sub',
 'D_105_lag1_sub',
 'D_107_lag1_sub',
 'D_109_lag1_sub',
 'D_112_lag1_sub',
 'D_113_lag1_sub',
 'D_115_lag1_sub',
 'D_118_lag1_sub',
 'D_119_lag1_sub',
 'D_121_lag1_sub',
 'D_122_lag1_sub',
 'D_123_lag1_sub',
 'D_124_lag1_sub',
 'D_125_lag1_sub',
 'D_127_lag1_sub',
 'D_128_lag1_sub',
 'D_129_lag1_sub',
 'D_130_lag1_sub',
 'D_131_lag1_sub

In [4]:
#tune all to float16
%run "10_Data into model format.ipynb"
dataset = 'test'
for dataset in ['train','test']:
    file =dataset_list[0]
    temp_df = pd.read_feather(Package_path + file +'_'+dataset+'.ftr').set_index('customer_ID')
    temp_df = temp_df.astype('float16')
    temp_df = temp_df.reset_index()
    temp_df.to_feather(Package_path + file +'_'+dataset+'.ftr')


function: load_DProcessed_df(file), amex_metric(y_true, y_pred), file_get(dataset,version,dataset_list,num_list_inModel,ignore_list)
list:all_cols,cat_list,num_list , varible: Huge_data_path
